Contains all the code for generating figures in the publication. Requires filtered NanoSeq mutations, callable coverage arrays, and MA line mutations and 1001 variants (from reformatting_ma_line_muts.ipynb)

In [ ]:
import sys
import os
sys.path.append(os.getcwd() + '/../python_scripts') # this lets us import files in python_scripts (like gtools)
import gtools
if os.getcwd()[:8] != '/scratch': # switch to the directory where all the data files are
    os.chdir(f'/scratch/cam02551/{os.getcwd().split("/")[-2]}')

import pandas as pd
import matplotlib.pyplot as plt
import csv
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import pysam
import subprocess
import importlib
import random
%matplotlib inline
import matplotlib.patches as patches
import pickle
import math
import scipy.stats as stats
import scipy.cluster as cluster
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
sample_names = ''
sample_names += 'untreated_1 untreated_2 untreated_3 untreated_4 untreated_5 untreated_6 untreated_7 untreated_8 '
# sample_names += 'UVB_1 UVB_2 UVC_002 UVC_005 UVC_01 UVC_02 '
# sample_names += 'Ler-0_1 '
sample_names = sample_names.split()

mut_files = [f'data/variant/{s}_merged_filtered.tsv' for s in sample_names] # filtered mutations from snakemake pipeline
cov_prefixes = {s:f'data/coverage/{s}_merged_duplex/' for s in sample_names} # callable coverage array prefixes from snakemake pipeline
genome_fasta = 'data/ref/ref.fa'
# conflict rates can be found in the stdout of duplex_metadata.py
conflict_rate_dict = {'untreated_1_merged.bam': 0.1421243306017736, 'untreated_2_merged.bam': 0.07499291504028818, 'untreated_3_merged.bam': 0.06698923466207254, 'untreated_4_merged.bam': 0.08825301441514141, 'untreated_5_merged.bam': 0.10694477979171899, 'untreated_6_merged.bam': 0.06708132263901942, 'untreated_7_merged.bam': 0.09039258234998046, 'untreated_8_merged.bam': 0.1131217594141343, 'UVB_1_merged.bam': 0.005914724117601422, 'UVB_2_merged.bam': 0.02487706726801387, 'UVC_002_merged.bam': 0.007101074947091077, 'UVC_005_merged.bam': 0.012911711246730054, 'UVC_01_merged.bam': 0.005330398460813733, 'UVC_02_merged.bam': 0.014511010508950055, 'Ler-0_1_merged.bam': 0.03273057807000435}
ma_line_dir = 'data/variant/weng_split/' # directory containing a tsv variant file for each MA line (generated in reformatting_ma_line_muts.ipynb)
polymorph_file = 'data/variant/1001_muts.tsv' # 1001 genomes polymorphisms (generated in reformatting_ma_line_muts.ipynb)

In [ ]:
conflict_rates = [conflict_rate_dict[s + '_merged.bam'] for s in sample_names]

In [ ]:
os.makedirs('figs/', exist_ok=True)

In [ ]:
genotypes = list({s.rsplit('_', 1)[0]:0 for s in sample_names})

# convert a sample name to its genotype name
def s_to_g(sample):
    return sample.rsplit('_', 1)[0]

def g_to_s(genotype):
    return [s for s in sample_names if s_to_g(s) == genotype]

In [ ]:
def gen_cmap(gen):
    gen_to_i = {'untreated':0, 'UVB':2, 'UVC':3, 'MA lines':6, 'Col-0-swapped':11}
    if gen in gen_to_i:
        return gtools.base_cmap[gen_to_i[gen]]
    
    gen_to_i = {'Ler-0':8, '1001':9}
    if gen in gen_to_i:
        return gtools.light_cmap[gen_to_i[gen]]

    return 'grey'

In [ ]:
# load genome fasta
genome = gtools.load_genome(genome_fasta)
chrom_sizes = {chrom:len(genome[chrom]) for chrom in genome if chrom != 'ChrC' and chrom != 'ChrM'}

# Load the mutations
remove duplicates, non-nuclear muts, and contaminants

In [ ]:
# load the filtered mutations, discard duplicates and non-nuclear contigs
dfs_mut = dict()
for i, file in enumerate(mut_files):
    df = pd.read_table(file, quoting=csv.QUOTE_NONE)
    raw_num = len(df)
    
    df = df.drop_duplicates('chrom pos ref alt'.split())
    unique_num = len(df)
    
    df = df[(df.chrom != 'ChrC') & (df.chrom != 'ChrM')]
    df['sample_freq'] = df.sample_sup / df.sample_cov
    df.sample_freq = df.sample_freq.fillna(0)
    print(f'{sample_names[i]}: {raw_num} muts loaded, {unique_num} unique, {len(df)} nuclear' + (f', WARNING {sum(df.sample_cov == 0)} with undefined frequency set to 0' if sum(df.sample_cov == 0) > 0 else ''))
    
    dfs_mut[sample_names[i]] = df

In [ ]:
# label variants which are present in a sibling (same genotype)
for g in genotypes:
    for s1 in g_to_s(g):
        in_sibling = np.zeros(len(dfs_mut[s1]), dtype=bool)
        for s2 in g_to_s(g):
            if s1 == s2:
                continue
            df = dfs_mut[s1].merge(dfs_mut[s2], how='left', on='chrom pos ref alt'.split(), indicator='_merge')
            in_sibling = in_sibling + np.array(df._merge == 'both')
        dfs_mut[s1]['in_sibling'] = in_sibling
        print(f'{s1}: {sum(in_sibling)} muts in a sibling')

In [ ]:
# for each pair of samples, get all the "contaminant" mutations found in both
dfs_contam = []
for sample_1 in sample_names:
    for sample_2 in sample_names:
        if s_to_g(sample_1) == s_to_g(sample_2): # ignore siblings
            continue
        if len(dfs_mut[sample_1]) > 10000 or len(dfs_mut[sample_2]) > 10000: # ignore samples with large numbers of mutations (the Ler-0 germline mutations)
            continue
        if 'swapped' in sample_1 or 'swapped' in sample_2: # ignore swapped samples
            continue
        
        df = dfs_mut[sample_1].merge(dfs_mut[sample_2]['chrom pos alt ref'.split()], how='inner')
        if len(df) > 0:
            # print(f'{sum(df.in_sibling)} germline and {sum(~df.in_sibling)} somatic mutations in {sample_1} are in {sample_2}')
            dfs_contam.append(df.copy())
df_contam = pd.concat(dfs_contam) if len(dfs_contam) > 0 else pd.DataFrame(columns='chrom pos ref alt'.split())
df_contam = df_contam.sort_values('chrom pos ref alt'.split())
contaminants = set(zip(df_contam.chrom, df_contam.pos, df_contam.ref, df_contam.alt))
print(f'{len(contaminants)} unique and {len(df_contam)} total contaminant mutations')

In [ ]:
# remove all contaminant mutations
for s in sample_names:
    if 'swapped' in s or len(dfs_mut[s]) > 10000: # don't discard "contaminants" for the swapped samples or samples with a large number of muts
        continue

    before_num = len(dfs_mut[s])
    dfs_mut[s] = dfs_mut[s][dfs_mut[s].apply(lambda r: (r.chrom, r.pos, r.ref, r.alt) not in contaminants, axis=1)]
    print(f'{s}: removed {before_num - len(dfs_mut[s])} contaminants')

# Separate somatic and germline muts

In [ ]:
dfs_mut_g = {g:None for g in genotypes}
for s in dfs_mut:
    if dfs_mut_g[s_to_g(s)] is None:
        dfs_mut_g[s_to_g(s)] = dfs_mut[s].copy()
    else:
        dfs_mut_g[s_to_g(s)] = pd.concat([dfs_mut_g[s_to_g(s)], dfs_mut[s]])

In [ ]:
vis_genotypes = genotypes

# plot histograms of mutation freq, color based on presence in sibling or another fragment
# bins = np.append(np.linspace(0, 0.014, 15), np.linspace(0.015, 1.015, 25))
bins = np.logspace(np.log10(0.001), np.log10(1), 30)
breakpoint = 15
fig, axs = plt.subplots(math.ceil(len(vis_genotypes) / 3), min(len(vis_genotypes), 3), sharex=True, figsize=(4 * min(len(vis_genotypes), 3), 0.5 * math.ceil((len(vis_genotypes) + 3) / 3)))
for i, g in enumerate(vis_genotypes):
    df = dfs_mut_g[g]
    if len(vis_genotypes) == 1:
        ax = axs
    elif len(vis_genotypes) <= 3:
        ax = axs[i % 3]
    else:
        ax = axs[i // 3][i % 3]
    cmap = plt.get_cmap('tab20')
    
    hml = ax.hist(df[~df.in_sibling & (df.sample_sup > 2) & (df.sample_freq < 0.35)].sample_freq, bins=bins, rwidth=0.85, color=cmap(2))
    hmh = ax.hist(df[~df.in_sibling & (df.sample_sup > 2) & (df.sample_freq >= 0.35)].sample_freq, bins=bins, rwidth=0.85, color=cmap(3), bottom=hml[0])
    hul = ax.hist(df[~df.in_sibling & (df.sample_sup <= 2) & (df.sample_freq < 0.35)].sample_freq, bins=bins, rwidth=0.85, color=cmap(0), bottom=hml[0] + hmh[0])
    huh = ax.hist(df[~df.in_sibling & (df.sample_sup <= 2) & (df.sample_freq >= 0.35)].sample_freq, bins=bins, rwidth=0.85, color=cmap(1), bottom=hml[0] + hmh[0] + hul[0])
    hs = ax.hist(df[df.in_sibling].sample_freq, bins=bins, rwidth=0.85, color=cmap(5), bottom=hul[0] + huh[0] + hml[0] + hmh[0])
    
    ax.set_xscale('log')
    ax.set_title(g, y=0.45)
ax.set_xlabel('Frequency')
fig.legend([hul[2], huh[2], hml[2], hmh[2], hs[2]], ['1 fragment', '1 framgnet (filtered as germline)', '>1 fragment', '>1 fragment (filtered as germline)', 'in sibling (filtered as germline)'], bbox_to_anchor=(0.9, 0.91), loc='lower right')
fig.suptitle('Histogram of mutation frequencies')
fig.savefig('figs/hist_of_mutation_frequencies.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# identify somatic mutations as those not present in a sibling, and at low frequency
dfs_somatic = dict()
for sample in dfs_mut:
    df = dfs_mut[sample]
    if sum(df.sample_freq.isna()) > 0:
        print(f'WARNING: {sum(df.sample_freq.isna())} mutations in {sample} with undefined frequency, assigning these a frequency of 0')
        df.sample_freq = df.sample_freq.fillna(0)
    if 'swapped' in sample: # don't discard swapped sample muts if in a "sibling"
        dfs_somatic[sample] = df[df.sample_freq < 0.35].copy()
    elif 'Ler-0' in sample: # only keep high abundance Ler-0 mutations
        dfs_somatic[sample] = df[df.sample_freq > 0.35].copy()
    else: # keep only low-abundance unique mutations as somatic
        dfs_somatic[sample] = df[~df.in_sibling & (df.sample_freq < 0.35)].copy()
    print(f'{sample}: kept {len(dfs_somatic[sample])} of {len(df)} mutations as "somatic"')
    
# # identify germline mutations as those which are present in a sibling
# dfs_germline = dict()
# for sample in dfs_mut:
#     df = dfs_mut[sample]
#     dfs_germline[sample] = df[df.in_sibling | (df.sample_freq >= 0.35)].copy()

# add the MA line and 1001 polymorphism data

In [ ]:
if 'MA lines' not in genotypes:
    for file in os.listdir(ma_line_dir):
        df = pd.read_table(f'{ma_line_dir}{file}')
        df = df[df.alt != 'N']
        dfs_somatic[f'MA lines_{file.split("_")[1]}'] = df
        sample_names.append(f'MA lines_{file.split("_")[1]}')
        conflict_rates.append(0)
    genotypes += ['MA lines']
    print(f'loaded {sum([len(dfs_somatic[sample]) for sample in sample_names if s_to_g(sample) == "MA lines"])} MA line mutations')
    
    dfs_somatic['1001_0'] = pd.read_table(polymorph_file)
    sample_names.append('1001_0')
    genotypes.append('1001')
    conflict_rates.append(0)
    print(f'loaded {len(dfs_somatic["1001_0"])} 1001 mutations')
else:
    print('ERROR: rerunning this cell requires rerunning the header cells first')

# Load callable coverage data (always run, it alters the MA line and 1001 datasets)

In [ ]:
sample_call_cov = {sample:0 for sample in sample_names}
for chrom in chrom_sizes:
    for i, sample in tqdm(enumerate(sample_names), total=len(sample_names)):
        if 'MA lines' in sample or '1001' in sample:
            if i == 0 or 'MA lines' not in sample_names[i - 1]: # if the last sample wasn't also an MA line sample, this if statement just saves time
                arr = np.load(f'{cov_prefixes['untreated_1']}{chrom}.npy')
                arr = arr > 0
            
            # remove any mutations in regions I'm defining as uncallable (this does discard about 300/2600 MA line mutations though)
            dfs_somatic[sample] = dfs_somatic[sample][(dfs_somatic[sample].chrom != chrom) | dfs_somatic[sample].pos.map(lambda x: arr[x] if x < len(arr) else True)]
        elif 'Ler-0' in sample:
            arr = np.load(f'{cov_prefixes[sample]}{chrom}.npy')
            arr = arr > 0
        else:
            arr = np.load(f'{cov_prefixes[sample]}{chrom}.npy')
        sample_call_cov[sample] += np.sum(arr) * (1 - conflict_rates[i]) # FIXME is it safe to do the conflict rate correction here?

In [ ]:
cov_samples = sample_names[:8]
callable_histogram = {sample:np.zeros(1000, dtype=int) for sample in cov_samples}
for sample in tqdm(cov_samples):
    for chrom in chrom_sizes:
        arr = np.load(f'{cov_prefixes[sample]}{chrom}.npy')
        callable_histogram[sample] += np.histogram(arr, bins=range(1001))[0]
print('number of sites with at least one callable coverage:')
for sample in cov_samples:
    print(f'{sample}: {np.sum(callable_histogram[sample][1:])}')

In [ ]:
fig, axs = plt.subplots(len(cov_samples), sharex=True)
x_max = 150
for i, sample in enumerate(cov_samples):
    axs[i].bar(range(x_max), callable_histogram[sample][:x_max])
    axs[i].set_title(sample, y=0.3, loc='right')
axs[-1].set_xlabel('Callable coverage')
axs[len(axs) // 2].set_ylabel('Count (genomic sites)')
plt.show()
fig.savefig('figs/callable_coverage_histogram.png', dpi=300, bbox_inches='tight')

# Save mutations for supplemental table

In [ ]:
# save mutations
df_supp_table = pd.DataFrame(columns='sample chrom pos ref alt'.split())
for s in sample_names:
    if s_to_g(s) not in ['Col-0', 'UVB', 'UVC']:
        continue
    df = dfs_somatic[s]['chrom pos ref alt'.split()].copy()
    correct_name = f'untreated-{s[-1]}' if s_to_g(s) == 'Col-0' else s
    df['sample'] = [correct_name] * len(df)
    df_supp_table = pd.concat([df_supp_table, df])

df_supp_table.to_csv('data/supp_table.tsv', sep='\t', index=False)

# Investigate mutation frequency

In [ ]:
ignore_samples = ['Ler-0', 'MA lines', '1001']
vis_samples = [s for s in sample_names if sum([x in s for x in ignore_samples]) == 0]
vis_genotypes = list({s_to_g(s):0 for s in vis_samples})

# TODO I need something that conveys what this plot is trying to do while correcting for coverage
# count the number of mutants observed at least 1, 2, 3, 4, and 5 times
max_obs = 5
obs_sample_counts = np.zeros((max_obs, len(vis_samples))) # obs_rates[number of observations][sample index] = fraction of mutations observed at least that many times + 1
obs_genotype_counts = np.zeros((max_obs, len(vis_genotypes))) 
for i in range(max_obs):
    for j, sample in enumerate(vis_samples):
        obs_sample_counts[i][j] = sum((dfs_somatic[sample].sample_sup / 2) > i)
        obs_genotype_counts[i][vis_genotypes.index(s_to_g(sample))] += sum((dfs_somatic[sample].sample_sup / 2) > i)

obs_sample_rates = obs_sample_counts / obs_sample_counts[0]
obs_genotype_rates = obs_genotype_counts / obs_genotype_counts[0]

        
fig, ax = plt.subplots(figsize=(12, 3))
# h = np.zeros(len(vis_samples))
for i in range(max_obs):
    ax.bar(range(len(vis_samples)), obs_sample_rates[i]) # don't plot ler
    # h += obs_sample_rates[i]
ax.legend([f'>={max_obs} observations' if i == max_obs else f'{i} observation(s)' for i in range(1, max_obs + 1)], loc='upper left', bbox_to_anchor=(0, 1))
ax.set_xticks(range(len(vis_samples)))
# ax.set_ylim(0, 0.3)
ax.set_xticklabels(vis_samples, rotation=90)
ax.set_title('Fraction of somatic mutations observed in multiple fragments')
fig.savefig('figs/sfs_stacked_bar_samples.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
h = np.zeros(len(vis_genotypes))
for i in range(max_obs):
    ax.bar(range(len(vis_genotypes)), obs_genotype_rates[i], bottom=h)
    h += obs_genotype_rates[i]
ax.legend([f'>={max_obs} observations' if i == max_obs else f'{i} observation(s)' for i in range(1, max_obs + 1)])
ax.set_xticks(range(len(vis_genotypes)), labels=vis_genotypes)
ax.set_title('Fraction of somatic mutations observed in multiple fragments')
fig.savefig('figs/sfs_stacked_bar_genotypes.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib
fig, ax = plt.subplots()
for c, genotype in enumerate(vis_genotypes):
    x = []
    y = []
    for sample in vis_samples:
        if s_to_g(sample) == genotype:
            x.append(sample_call_cov[sample])
            y.append(sum(dfs_somatic[sample].sample_sup > 2) / sum(dfs_somatic[sample].sample_cov > 2))
    ax.scatter(x, y, color=gen_cmap(c))
ax.legend(vis_genotypes, loc='upper left', bbox_to_anchor=(1, 1))
ax.set_xlabel('Callable coverage')
ax.set_ylabel('Fraction of somatic mutations\nobserved in >1 fragment')
ax.set_title('Fraction "high frequency" somatic mutations')
fig.savefig('figs/frac_high_freq_somatic_mutations_call_cov.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib
fig, ax = plt.subplots()
for c, genotype in enumerate(vis_genotypes):
    x = []
    y = []
    for sample in vis_samples:
        if s_to_g(sample) == genotype:
            x.append(len(dfs_somatic[sample]) / sample_call_cov[sample])
            y.append(sum(dfs_somatic[sample].sample_sup > 2) / sum(dfs_somatic[sample].sample_cov > 2))
    ax.scatter(x, y, color=gen_cmap(c))
ax.legend(vis_genotypes, loc='upper left', bbox_to_anchor=(1, 1))
ax.set_xlabel('Somatic mutation rate')
ax.set_ylabel('Fraction of somatic mutations\nobserved in >1 fragment')
ax.set_title('Fraction "high frequency" somatic mutations')
fig.savefig('figs/frac_high_freq_somatic_mutations_mut_rate.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib
fig, ax = plt.subplots()
for c, genotype in enumerate(vis_genotypes):
    x = []
    y = []
    for sample in sample_names:
        if s_to_g(sample) == genotype:
            x.append(len(dfs_somatic[sample]))
            y.append(sum(dfs_somatic[sample].sample_sup > 2) / sum(dfs_somatic[sample].sample_cov > 2))
    ax.scatter(x, y, color=gen_cmap(c))
ax.legend(vis_genotypes, loc='upper left', bbox_to_anchor=(1, 1))
ax.set_xlabel('Somatic mutations detected')
ax.set_ylabel('Fraction of somatic mutations\nobserved in >1 fragment')
ax.set_title('Fraction "high frequency" somatic mutations')
fig.savefig('figs/frac_high_freq_somatic_mutations_mut_count.png', dpi=300, bbox_inches='tight')
plt.show()

# Somatic mutation rates

In [ ]:
# count the mutations and total coverage of each genotype (across all replicates)
gen_counts = {g:[] for g in genotypes}
gen_coverage = {g:[] for g in genotypes}
for s in sample_names:
    # add mutation count and coverage to the running totals for the genotype
    gen_counts[s_to_g(s)].append(len(dfs_somatic[s]))
    gen_coverage[s_to_g(s)].append(sample_call_cov[s])
gen_count_sums = {g:sum(gen_counts[g]) for g in genotypes}
gen_coverage_sums = {g:sum(gen_coverage[g]) for g in genotypes}
gen_rates = {g:gen_count_sums[g] / gen_coverage_sums[g] for g in genotypes}

In [ ]:
{g:sum(gen_counts[g]) for g in genotypes}

In [ ]:
# do a t-test for each genotype compared to untreated WT
p_ttest = dict()
col_rates = [gen_counts['untreated'][i] / gen_coverage['untreated'][i] for i in range(len(gen_coverage['untreated']))]
for g in genotypes:
    if len(gen_coverage[g]) == 1: # you can't do a t-test with one replicate
        p_ttest[g] = 1
        continue
    p_ttest[g] = stats.ttest_ind(col_rates, [gen_counts[g][i] / gen_coverage[g][i] for i in range(len(gen_coverage[g]))], equal_var=False)[1]

In [ ]:
# get 95% confidence intervals assuming t-distribution
t_yerr = [dict(), dict()]
for g in genotypes:
    if len(gen_coverage[g]) > 1:
        sem = np.std([gen_counts[g][i] / gen_coverage[g][i] for i in range(len(gen_coverage[g]))]) / (len(gen_coverage[g]) ** (1/2))
        
        low, high = stats.t.interval(0.95, len(gen_coverage[g]) - 1, loc=gen_rates[g], scale=sem)
    else:
        print(f'WARNING: no replicates for genotype {g}, using Poisson to calculate error bars')
        low, high = stats.poisson(gen_count_sums[g]).interval(0.95)
        low = low / gen_coverage_sums[g]
        high = high / gen_coverage_sums[g]
    t_yerr[0][g] = gen_rates[g] - low
    t_yerr[1][g] = high - gen_rates[g]

In [ ]:
sems = dict()
for g in genotypes:
    sem = np.std([gen_counts[g][i] / gen_coverage[g][i] for i in range(len(gen_coverage[g]))]) / (len(gen_coverage[g]) ** (1/2))
    sems[g] = sem

vis_genotypes = [g for g in genotypes if 'Ler-0' not in g and '1001' not in g]
vis_samples = [s for s in sample_names if s_to_g(s) in vis_genotypes]

cor_p_ttest = {g:p_ttest[g] * (len(vis_genotypes) - 1) for g in vis_genotypes}

max_rate = 1.5 * 10**-6
fig, axs = plt.subplots(2, figsize=(len(vis_genotypes), 6), sharex=True, gridspec_kw={'height_ratios':(1, 3)})
axs[0].bar(range(len(vis_genotypes)), [gen_count_sums[g] for g in vis_genotypes], color='tab:orange')
y = [min(max_rate, gen_count_sums[g] / gen_coverage_sums[g]) for g in vis_genotypes]
# yerr = [[t_yerr[0][g] for g in vis_genotypes], [t_yerr[1][g] for g in vis_genotypes]]
yerr = [sems[g] for g in vis_genotypes]
bars = axs[1].bar(range(len(vis_genotypes)), y, yerr=yerr, color=[gen_cmap(g) for g in vis_genotypes], alpha=0.6)
for i, g in enumerate(vis_genotypes):
    axs[1].scatter([i + random.random() * 0.5 - 0.25 for x in gen_counts[g]], [min(max_rate, gen_counts[g][j] / gen_coverage[g][j]) for j in range(len(gen_counts[g]))], color=gen_cmap(g), edgecolors='none')
    text = f'p<e{math.ceil(math.log10(max(10 ** -50, cor_p_ttest[g])))}' if cor_p_ttest[g] < 10 ** -3 else f'\np={cor_p_ttest[g]:.3f}'
    # axs[1].text(i, min(bars[i].get_height() * 1.1, max_rate), text, ha='center', fontsize=12)
axs[1].set_xticks(range(len(vis_genotypes)))
axs[1].set_xticklabels(vis_genotypes, rotation=45, fontsize=14, ha='right')
axs[0].set_title('Somatic mutation counts by genotype', y=1)
axs[1].set_title('Somatic mutation rates by genotype', y=1)
axs[1].set_ylabel('Somatic mutations / callable bp')
# axs[1].set_yscale('log')
# axs[1].set_ylim(axs[1].get_ylim()[0], axs[1].get_ylim()[1] * 2.5)
# axs[1].set_ylim(0, 6 * 10**-7)
xlim = axs[1].get_xlim()
axs[1].hlines(bars[0].get_height(), -1, len(gen_counts) + 1, color='black')
# axs[1].hlines(8.25 * 10 ** -9, -1, len(gen_counts) + 1, color='tab:red')
axs[1].set_xlim(xlim)
fig.savefig('figs/somatic_mutation_rates_by_genotype.svg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots()
for i in range(len(rates)):
    ax.plot(target_muts, p_power[i])
xlim = ax.get_xlim()
ax.hlines(0.05, -100, 10000, color='black')
ax.hlines(0.0025, -100, 10000, color='black')
ax.set_xlim(xlim)
ax.legend(rates, bbox_to_anchor=(1, 1), title='Mutation rate\nincrease over\nCol-0')
ax.set_yscale('log')
ax.set_ylim(0.00001, 0.1)
ax.set_title('Mutations needed to detect an increase over Col-0')
ax.set_xlabel('Targeted number of mutations')
ax.set_ylabel('Chi squared P-value (power 0.9)')
fig.savefig('figs/mutations_needed_to_detect_an_increase_over_col-0.png', dpi=300, bbox_inches='tight')

In [ ]:
df_wt = pd.concat(dfs_somatic[:10])
df_ner = pd.concat(dfs_somatic[10:])
len(df_wt), len(df_ner)

In [ ]:
genome = gtools.load_genome(genome_fasta)

In [ ]:
# turn the mutation counts into a bedgraph
with open('tmp/wt_muts.bg', 'w') as f:
    for chrom in genome:
        positions = list(df_wt[df_wt.chrom == chrom].pos)
        unique_pos = sorted(list(set(positions)))
        last_pos = -1
        for pos in unique_pos:
            if pos != 0: # avoids writing a [0, 0) block if there's a mutation at position 0 
                f.write(f'{chrom}\t{last_pos + 1}\t{pos}\t0\n')
            f.write(f'{chrom}\t{pos}\t{pos + 1}\t{positions.count(pos)}\n') # this won't merge two mutations at adjacent positions into one block
            last_pos = pos
        if last_pos + 1 != len(genome[chrom]): # avoids writing a [chrom len, chrom len) block if there's a mutation at the last base
            f.write(f'{chrom}\t{last_pos + 1}\t{len(genome[chrom])}\t0\n')

with open('tmp/ner_muts.bg', 'w') as f:
    for chrom in genome:
        positions = list(df_ner[df_ner.chrom == chrom].pos)
        unique_pos = sorted(list(set(positions)))
        last_pos = -1
        for pos in unique_pos:
            if pos != 0:
                f.write(f'{chrom}\t{last_pos + 1}\t{pos}\t0\n')
            f.write(f'{chrom}\t{pos}\t{pos + 1}\t{positions.count(pos)}\n') # this won't merge two mutations at adjacent positions into one block
            last_pos = pos
        if last_pos + 1 != len(genome[chrom]):
            f.write(f'{chrom}\t{last_pos + 1}\t{len(genome[chrom])}\t0\n')

In [ ]:
# make callable coverage arrays summing over wt or ner mutant samples
wt_cov = None
for d in list(call_cov.values())[:10]:
    if wt_cov is None:
        wt_cov = d.copy()
    else:
        for chrom in d:
            wt_cov[chrom] += d[chrom]

ner_cov = None
for d in list(call_cov.values())[10:]:
    if ner_cov is None:
        ner_cov = d.copy()
    else:
        for chrom in d:
            ner_cov[chrom] += d[chrom]

In [ ]:
# turn the callable coverage arrays into bedgraphs
with open('tmp/wt_cov.bg', 'w') as f:
    for chrom in wt_cov:
        start = 0
        value = 0
        for i, cov in tqdm(enumerate(wt_cov[chrom]), total=len(wt_cov[chrom])):
            if cov != value:
                if i != 0:
                    f.write(f'{chrom}\t{start}\t{i}\t{value}\n')
                start = i
                value = cov
        f.write(f'{chrom}\t{start}\t{len(genome[chrom])}\t{value}\n')
    f.write(f'ChrC\t0\t154478\t0\n')
    f.write(f'ChrM\t0\t367808\t0\n')
        
with open('tmp/ner_cov.bg', 'w') as f:
    for chrom in ner_cov:
        start = 0
        value = 0
        for i, cov in tqdm(enumerate(ner_cov[chrom]), total=len(ner_cov[chrom])):
            if cov != value:
                if i != -1:
                    f.write(f'{chrom}\t{start}\t{i}\t{value}\n')
                start = i
                value = cov
        f.write(f'{chrom}\t{start}\t{len(genome[chrom])}\t{value}\n')
    f.write(f'ChrC\t0\t154478\t0\n')
    f.write(f'ChrM\t0\t367808\t0\n')

# Mutation spectra

In [ ]:
# find every possible mutation 3bp context e.g. ACAT means ACA->ATA
contexts = []
for mut in ['CA', 'CG', 'CT', 'TA', 'TC', 'TG']: # SNP type e.g. A->C
    for bef in 'ACGT':
        for aft in 'ACGT':
            contexts.append(f'{bef}{mut[0]}{aft}{mut[1]}')
contexts.append('del')
contexts.append('ins')

In [ ]:
# reverse complement a sequence
comp = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
def rc(seq):
    ret = ''
    for nuc in reversed(seq):
        ret += comp[nuc]
    return ret

In [ ]:
# restrict_region = None
# restrict_region = region_arrays['exon']

# generate a null mutation spectrum
# load callable coverage of first sample
call_cov_0 = dict()
call_cov_0_chroms = dict()
for chrom in chrom_sizes:
    call_cov_0[chrom] = np.load(f'{cov_prefixes["untreated_1"]}{chrom}.npy')
    call_cov_0_chroms[chrom] = np.sum(call_cov_0[chrom])

# randomly sample positions in the genome based on callable coverage
sample_size = 2000000
sampled_positions = dict()
for chrom in tqdm(chrom_sizes):
    k = int(sample_size * np.sum(call_cov_0[chrom]) / sum([np.sum(call_cov_0[c]) for c in chrom_sizes]))
    sampled_positions[chrom] = random.choices(range(len(call_cov_0[chrom])), weights=call_cov_0[chrom], k=k)

# build a null spectrum using the randomly sampled positions, won't include SNP info
context_freq = {c[0:3]:0 for c in contexts}
for chrom in tqdm(chrom_sizes):
    for pos in sampled_positions[chrom]:
        context = genome[chrom][pos - 1:pos + 2]
        if context[0] not in 'ACGT' or context[1] not in 'ACGT' or context[2] not in 'ACGT': # skip context with N, W, R, Y...
            continue
        if context[1] == 'A' or context[1] == 'G': # reverse complement contexts centered on G or T
            context = rc(context)
        context_freq[context] += 1
context_freq = {c:context_freq[c] / sum(context_freq.values()) for c in context_freq} # so they sum to 1
context_freq['ins'] = 1
context_freq['del'] = 1

In [ ]:
# calculate the spectrum for each sample
spectra = {sample:{c:0 for c in contexts} for sample in sample_names}
for sample in sample_names:
    warned = False
    for r in dfs_somatic[sample].itertuples():
        if r.ref == '*':
            spectra[sample]['ins'] += 1
        elif r.alt == '*':
            spectra[sample]['del'] += 1
        else:
            alt = r.alt
            context = genome[r.chrom][r.pos - 1:r.pos + 2]
            if not set(context).issubset({'A', 'C', 'G', 'T'}):
                print(f'WARNING: context of {context} in sample {sample} for mutation {r}, ignoring this mutation')
                continue
            if context[1] != r.ref:
                context = context[0] + r.ref + context[2]
                if not warned:
                    print(f'WARNING: ref base does not match reference genome in sample {sample} for mutation {r}, hiding further warnings for this sample')
                    warned = True
            if r.ref == 'A' or r.ref == 'G': # if the mut has the reference as G or T, reverse copmlement the context
                context = rc(context)
                alt = comp[r.alt]
            spectra[sample][context + alt] += 1


In [ ]:
# normalize by frequency of sequence context, total callable coverage, and avg genome-wide mutation rate, the value is now the mutation rate relative to genome-wide average
# norm rate = num mutations in context / (frequency of context * total coverage) / (total mutations / total coverage)
#                                                 ^coverage of context                 ^genome-wide avg mutation rate
# norm rate = num mutations in context / frequency of context / total mutations
def normalize_spectrum(spec, spec_freq, total_mutations):
    s = dict()
    for c in spec:
        freq_c = c[:-1] if c != 'ins' and c != 'del' else c
        s[c] = spec[c] / spec_freq[freq_c] / total_mutations
    return s

norm_spectra = dict()
for sample, spec in spectra.items():
    norm_spectra[sample] = normalize_spectrum(spec, context_freq, sum(spec.values()))

In [ ]:
# ignore_samples = []
# vis_samples = sample_names[:8]
# vis_genotypes = list({s_to_g(s):0 for s in vis_samples})

# mut_colors = ['tab:cyan'] * 16 + ['black'] * 16 + ['tab:red'] * 16 + ['tab:grey'] * 16 + ['tab:green'] * 16 + ['tab:pink'] * 16 + ['tab:purple'] + ['tab:brown']

# fig, axs = plt.subplots(len(vis_samples), figsize=(14, 0.7 * len(vis_samples)), sharex=True, sharey=True)
# for i, sample in enumerate(vis_samples):
#     axs[i].bar(range(len(contexts)), norm_spectra[sample].values(), color=mut_colors)
#     title = sample + ' (n=' + str(len(dfs_somatic[sample])) + ')'
#     axs[i].set_title(title, y=0.5, x=0.01, loc='left')
#     axs[i].ticklabel_format(style='plain')
# axs[-1].set_xticks(range(len(contexts)))
# axs[-1].set_xticklabels([c[:3] for c in spectra[vis_samples[0]].keys()], rotation=90, size=6)
# for pos, label in zip([7.5 + 16 * i for i in range(6)], ['C -> A', 'C -> G', 'C -> T', 'T -> A', 'T -> C', 'T -> G']): # trying to set separate major and minor ticks has a weird bug
#     axs[-1].text(pos, -1 * axs[-1].get_ylim()[1], label)
# axs[len(axs) // 2].set_ylabel('Mutation rate relative to avg')
# # axs[-1].set_ylim(0, 0.15)
# fig.suptitle('Spectra of somatic mutations', y=0.9)
# fig.savefig('figs/mutation_spectra_by_sample.png', dpi=300, bbox_inches='tight')

In [ ]:
# group spectra from similar treatments and sum them for a new plot

# sum the spectra of all samples for a given genotype
grouped_spectra = {g:{c:0 for c in contexts} for g in genotypes}
for sample in sample_names:
    genotype = s_to_g(sample)
    for c in contexts:
        grouped_spectra[genotype][c] += spectra[sample][c]

# make a 95% confidence interval for each context using biniomial distribution
gs_lower = {g:dict() for g in genotypes}
gs_upper = {g:dict() for g in genotypes}
for genotype in genotypes:
    for c in contexts:
        total = sum(grouped_spectra[genotype].values())
        lower, upper = stats.binom.interval(0.95, total, max(1, grouped_spectra[genotype][c]) / total)
        gs_lower[genotype][c] = lower
        gs_upper[genotype][c] = upper

# normalize the genotype spectra and confidence intervals; convert confidence intervals to error amounts
norm_grouped_spectra = dict()
ngs_yerr_0 = dict()
ngs_yerr_1 = dict()
for genotype in genotypes:
    norm_grouped_spectra[genotype] = normalize_spectrum(grouped_spectra[genotype], context_freq, sum(grouped_spectra[genotype].values()))
    lower = normalize_spectrum(gs_lower[genotype], context_freq, sum(grouped_spectra[genotype].values()))
    upper = normalize_spectrum(gs_upper[genotype], context_freq, sum(grouped_spectra[genotype].values()))
    ngs_yerr_0[genotype] = {c:abs(norm_grouped_spectra[genotype][c] - lower[c]) for c in contexts}
    ngs_yerr_1[genotype] = {c:abs(norm_grouped_spectra[genotype][c] - upper[c]) for c in contexts}


# count the number of mutations in each genotype
grouped_counts = {genotype: sum(grouped_spectra[genotype].values()) for genotype in genotypes}

In [ ]:
full_contexts = [c for c in contexts if c != 'ins' and c !='del']

sems = {g:dict() for g in genotypes}
for g in genotypes:
    for context in full_contexts:
        sems[g][context] = np.std([norm_spectra[s][context] for s in g_to_s(g)]) / (len(g_to_s(g)) ** (1/2))

# ignore_samples = []
vis_genotypes = genotypes
vis_samples = [s for s in sample_names if s_to_g(s) in vis_genotypes]

fig, axs = plt.subplots(len(vis_genotypes), figsize=(11, len(vis_genotypes) * 1.5), sharex=True, sharey=True)
full_colors = ['tab:cyan'] * 16 + ['black'] * 16 + ['tab:red'] * 16 + ['tab:grey'] * 16 + ['tab:green'] * 16 + ['tab:pink'] * 16
for i, genotype in enumerate(vis_genotypes):
    samples = g_to_s(genotype)
    # yerrs = [[ngs_yerr_0[genotype][c] for c in full_contexts], [ngs_yerr_1[genotype][c] for c in full_contexts]]
    axs[i].bar(range(len(full_contexts)), [norm_grouped_spectra[genotype][c] for c in full_contexts], color=full_colors, alpha=0.6, yerr=sems[genotype].values())
    # axs[i].errorbar([x + 0.1 for x in range(len(full_contexts))], [norm_grouped_spectra[genotype][c] for c in full_contexts], yerr=list(sems[genotype].values()), fmt='none')
    title = genotype + ' (n=' + str(grouped_counts[genotype]) + ')'
    if len(g_to_s(genotype)) > 1 and len(g_to_s(genotype)) < 30:
        for sample in g_to_s(genotype):
            size = 2 if len(g_to_s(genotype)) < 20 else 0.5
            axs[i].scatter([x + random.random() * 0.4 - 0.2 for x in range(len(full_contexts))], [norm_spectra[sample][c] for c in full_contexts], color=full_colors, s=size, edgecolors='none')
    axs[i].set_title(title, y=0.55, x=0.01, loc='left')
    axs[i].ticklabel_format(style='plain')

axs[-1].set_xticks(range(len(full_contexts)))
axs[-1].set_xticklabels([c[:3] for c in full_contexts], rotation=90, size=6)
axs[-1].set_ylim(0, 5.8)
for pos, label in zip([7.5 + 16 * i for i in range(6)], ['C -> A', 'C -> G', 'C -> T', 'T -> A', 'T -> C', 'T -> G']): # trying to set separate major and minor ticks has a weird bug
    axs[-1].text(pos, -1 * axs[-1].get_ylim()[1], label)
axs[len(axs) // 2].set_ylabel('mutation/polymorphism rate relative to uniform')
fig.suptitle('Grouped spectra of somatic mutations', y=0.92)
fig.savefig('figs/mutation_spectra_by_sample_groups.svg', dpi=300, bbox_inches='tight')
plt.show()

## Simple spectra

In [ ]:
# make the above plot, but simplify to just type of SNP (no 3bp context)
simple_contexts = ['CA', 'CG', 'CT', 'TA', 'TC', 'TG', 'del', 'ins']

# make a simple version of context_freq
simple_context_freq = {'C':0, 'T':0, 'del':0, 'ins':0}
for c in context_freq:
    simple = c if c == 'ins' or c == 'del' else c[1]
    simple_context_freq[simple] += context_freq[c]

# make simple versions of the genotype spectra
simple_spectra = {g:{c:0 for c in simple_contexts} for g in genotypes}
simple_spectra_samples = {s:{c:0 for c in simple_contexts} for s in sample_names}
for sample in sample_names:
    genotype = s_to_g(sample)
    for c in contexts:
        simple_c = c if c == 'ins' or c == 'del' else c[1] + c[3]
        simple_spectra[genotype][simple_c] += spectra[sample][c]
        simple_spectra_samples[sample][simple_c] += spectra[sample][c]

# make 95% confidence intervals of the simple spectra
ss_lower = {g:dict() for g in genotypes}
ss_upper = {g:dict() for g in genotypes}
for genotype in genotypes:
    for c in simple_contexts:
        total = sum(simple_spectra[genotype].values())
        lower, upper = stats.binom.interval(0.95, total, max(1, simple_spectra[genotype][c]) / total)
        ss_lower[genotype][c] = lower
        ss_upper[genotype][c] = upper

# normalize spectra and confidence intervals, convert confidence intervals to error amounts
norm_simple_spectra = dict()
norm_simple_spectra_samples = dict()
nss_yerr_0 = dict()
nss_yerr_1 = dict()
for genotype in genotypes:
    norm_simple_spectra[genotype] = normalize_spectrum(simple_spectra[genotype], simple_context_freq, sum(simple_spectra[genotype].values()))
    lower = normalize_spectrum(ss_lower[genotype], simple_context_freq, sum(simple_spectra[genotype].values()))
    upper = normalize_spectrum(ss_upper[genotype], simple_context_freq, sum(simple_spectra[genotype].values()))
    nss_yerr_0[genotype] = {c:abs(norm_simple_spectra[genotype][c] - lower[c]) for c in simple_contexts}
    nss_yerr_1[genotype] = {c:abs(norm_simple_spectra[genotype][c] - upper[c]) for c in simple_contexts}
for sample in sample_names:
    norm_simple_spectra_samples[sample] = normalize_spectrum(simple_spectra_samples[sample], simple_context_freq, sum(simple_spectra_samples[sample].values()))

In [ ]:
sems = {g:dict() for g in genotypes}
for g in genotypes:
    for context in simple_contexts:
        sems[g][context] = np.std([norm_simple_spectra_samples[s][context] for s in g_to_s(g)]) / (len(g_to_s(g)) ** (1/2))

fig, axs = plt.subplots(len(vis_genotypes), figsize=(4, len(vis_genotypes) * 1.5), sharex=True, sharey=True)
simple_colors = ['tab:cyan', 'black', 'tab:red', 'tab:grey', 'tab:green', 'tab:pink', 'tab:brown', 'tab:purple']
for i, genotype in enumerate(vis_genotypes):
    samples = g_to_s(genotype)
    yerrs = [list(nss_yerr_0[genotype].values()), list(nss_yerr_1[genotype].values())]
    axs[i].bar(range(len(simple_contexts)), norm_simple_spectra[genotype].values(), color=simple_colors, alpha=0.6, yerr=sems[genotype].values())
    title = genotype + ' (n=' + str(grouped_counts[genotype]) + ')'
    for sample in g_to_s(genotype):
        size = 8 if len(g_to_s(genotype)) < 20 else 1
        axs[i].scatter([x + random.random() * 0.5 - 0.25 for x in range(len(simple_contexts))], norm_simple_spectra_samples[sample].values(), color=simple_colors, s=size, edgecolors='none')
    axs[i].set_title(title, y=0.55, x=0.01, loc='left')
    axs[i].ticklabel_format(style='plain')

axs[-1].set_xticks(range(len(simple_contexts)))
axs[-1].set_xticklabels([c if c == 'ins' or c == 'del' else f'{c[0]}>{c[1]}' for c in simple_contexts], size=8)
axs[len(axs) // 2].set_ylabel('relative substitution rate')
axs[-1].set_ylim(0, axs[-1].get_ylim()[1])
fig.suptitle('Simple substitution spectra', y=0.92)
fig.savefig('figs/simple_mutation_spectra_by_sample_groups.svg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
for g in vis_genotypes:
    for context in simple_contexts:
        p = stats.ttest_ind([norm_simple_spectra_samples[s][context] for s in g_to_s(g)], [norm_simple_spectra_samples[s][context] for s in g_to_s('untreated')], equal_var=True).pvalue
        print(g, context, p * len(simple_contexts) * (len(vis_genotypes) - 1))

## CG and YC spectra

In [ ]:
# make the same plot as above, but for (C>T)G and (C>T)H sites

cg_contexts = ['NCGT', 'NCHT']

# get frequency of CG and CH sites
cg_freq = {'NCG':0, 'NCH':0}
for c in context_freq:
    if c == 'ins' or c == 'del':
        continue
    if c[1] == 'C' and c[2] == 'G':
        cg_freq['NCG'] += context_freq[c]
    elif c[1] == 'C':
        cg_freq['NCH'] += context_freq[c]

# get the counts of CG and CH mutations
cg_spectra = {g:{c:0 for c in cg_contexts} for g in genotypes}
cg_spectra_samples = {s:{c:0 for c in cg_contexts} for s in sample_names}
for sample in sample_names:
    genotype = s_to_g(sample)
    for c in contexts:
        if c[1] == 'C' and c[2] == 'G' and c[3] == 'T':
            cg_spectra[genotype]['NCGT'] += spectra[sample][c]
            cg_spectra_samples[sample]['NCGT'] += spectra[sample][c]
        elif c[1] == 'C' and c[3] == 'T':
            cg_spectra[genotype]['NCHT'] += spectra[sample][c]
            cg_spectra_samples[sample]['NCHT'] += spectra[sample][c]

# make 95% confidence intervals
cg_lower = {g:dict() for g in genotypes}
cg_upper = {g:dict() for g in genotypes}
for genotype in genotypes:
    for c in cg_contexts:
        total = sum(cg_spectra[genotype].values())
        lower, upper = stats.binom.interval(0.95, total, max(1, cg_spectra[genotype][c]) / total)
        cg_lower[genotype][c] = lower
        cg_upper[genotype][c] = upper

# normalize spectra and confidence intervals, convert confidence intervals to error amounts
norm_cg_spectra = dict()
norm_cg_spectra_samples = dict()
ncg_yerr_0 = dict()
ncg_yerr_1 = dict()
for genotype in genotypes:
    norm_cg_spectra[genotype] = normalize_spectrum(cg_spectra[genotype], cg_freq, sum(cg_spectra[genotype].values()))
    lower = normalize_spectrum(cg_lower[genotype], cg_freq, sum(cg_spectra[genotype].values()))
    upper = normalize_spectrum(cg_upper[genotype], cg_freq, sum(cg_spectra[genotype].values()))
    ncg_yerr_0[genotype] = {c:abs(norm_cg_spectra[genotype][c] - lower[c]) for c in cg_contexts}
    ncg_yerr_1[genotype] = {c:abs(norm_cg_spectra[genotype][c] - upper[c]) for c in cg_contexts}
for sample in sample_names:
    norm_cg_spectra_samples[sample] = normalize_spectrum(cg_spectra_samples[sample], cg_freq, sum(cg_spectra_samples[sample].values()))

In [ ]:
sems = {g:dict() for g in genotypes}
for g in genotypes:
    for context in cg_contexts:
        sems[g][context] = np.std([norm_cg_spectra_samples[s][context] for s in g_to_s(g)]) / (len(g_to_s(g)) ** (1/2))

fig, ax = plt.subplots(figsize=(len(vis_genotypes), 3))

c = gtools.light_cmap[0]
samples = g_to_s(genotype)
# yerrs = [[ncg_yerr_0[g]['NCGT'] for g in vis_genotypes], [ncg_yerr_1[g]['NCGT'] for g in vis_genotypes]]
yerrs = [sems[g]['NCGT'] for g in vis_genotypes]
bar_cg = ax.bar(range(0, len(vis_genotypes) * 3, 3), [norm_cg_spectra[genotype]['NCGT'] for genotype in vis_genotypes], yerr=yerrs, color=c, alpha=0.6)
ax.scatter([vis_genotypes.index(s_to_g(s)) * 3 + random.random() * 0.5 - 0.25 for s in vis_samples], [norm_cg_spectra_samples[s]['NCGT'] for s in vis_samples], color=c, s=[6 if len(g_to_s(s_to_g(s))) < 20 else 0.5 for s in vis_samples])

c = gtools.dark_cmap[0]
# yerrs = [[ncg_yerr_0[g]['NCHT'] for g in vis_genotypes], [ncg_yerr_1[g]['NCHT'] for g in vis_genotypes]]
yerrs = [sems[g]['NCHT'] for g in vis_genotypes]
bar_ch = ax.bar(range(1, len(vis_genotypes) * 3 + 1, 3), [norm_cg_spectra[genotype]['NCHT'] for genotype in vis_genotypes], yerr=yerrs, color=c, alpha=0.6)
ax.scatter([vis_genotypes.index(s_to_g(s)) * 3 + 1 + random.random() * 0.5 - 0.25 for s in vis_samples], [norm_cg_spectra_samples[s]['NCHT'] for s in vis_samples], color=c, s=[6 if len(g_to_s(s_to_g(s))) < 20 else 0.5 for s in vis_samples])

ax.set_xticks([x + 0.5 for x in range(0, len(vis_genotypes) * 3, 3)], labels=vis_genotypes)
ax.legend([bar_cg, bar_ch], ['(C>T)G', '(C>T)H'])
ax.set_ylim((0, 11.2))
fig.suptitle('CG site vs CH site C>T rate', y=0.95)
fig.savefig('figs/cg_vs_ch_spectrum.svg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
for g in vis_genotypes:
    cg = [norm_cg_spectra_samples[s]['NCGT'] for s in g_to_s(g)]
    ch = [norm_cg_spectra_samples[s]['NCHT'] for s in g_to_s(g)]
    p = stats.ttest_ind(cg, ch).pvalue
    print(g, norm_cg_spectra[g]['NCGT'] / norm_cg_spectra[g]['NCHT'], p * len(vis_genotypes))

In [ ]:
# make the same plot as above, but for Y(C>T) and R(C>T) sites

yc_contexts = ['YCNT', 'RCNT']

# get frequency of YC and RC sites
yc_freq = {'YCN':0, 'RCN':0}
for c in context_freq:
    if c == 'ins' or c == 'del':
        continue
    if (c[0] == 'C' or c[0] == 'T') and c[1] == 'C':
        yc_freq['YCN'] += context_freq[c]
    elif c[1] == 'C':
        yc_freq['RCN'] += context_freq[c]

# get the counts of YC and RC mutations
yc_spectra = {g:{c:0 for c in yc_contexts} for g in genotypes}
yc_spectra_samples = {s:{c:0 for c in yc_contexts} for s in sample_names}
for sample in sample_names:
    genotype = s_to_g(sample)
    for c in contexts:
        if (c[0] == 'C' or c[0] == 'T') and c[1] == 'C' and c[3] == 'T':
            yc_spectra[genotype]['YCNT'] += spectra[sample][c]
            yc_spectra_samples[sample]['YCNT'] += spectra[sample][c]
        elif c[1] == 'C' and c[3] == 'T':
            yc_spectra[genotype]['RCNT'] += spectra[sample][c]
            yc_spectra_samples[sample]['RCNT'] += spectra[sample][c]

# make 95% confidence intervals
yc_lower = {g:dict() for g in genotypes}
yc_upper = {g:dict() for g in genotypes}
for genotype in genotypes:
    for c in yc_contexts:
        total = sum(yc_spectra[genotype].values())
        lower, upper = stats.binom.interval(0.95, total, max(1, yc_spectra[genotype][c]) / total)
        yc_lower[genotype][c] = lower
        yc_upper[genotype][c] = upper

# normalize spectra and confidence intervals, convert confidence intervals to error amounts
norm_yc_spectra = dict()
norm_yc_spectra_samples = dict()
nyc_yerr_0 = dict()
nyc_yerr_1 = dict()
for genotype in genotypes:
    norm_yc_spectra[genotype] = normalize_spectrum(yc_spectra[genotype], yc_freq, sum(yc_spectra[genotype].values()))
    lower = normalize_spectrum(yc_lower[genotype], yc_freq, sum(yc_spectra[genotype].values()))
    upper = normalize_spectrum(yc_upper[genotype], yc_freq, sum(yc_spectra[genotype].values()))
    nyc_yerr_0[genotype] = {c:abs(norm_yc_spectra[genotype][c] - lower[c]) for c in yc_contexts}
    nyc_yerr_1[genotype] = {c:abs(norm_yc_spectra[genotype][c] - upper[c]) for c in yc_contexts}
for sample in sample_names:
    norm_yc_spectra_samples[sample] = normalize_spectrum(yc_spectra_samples[sample], yc_freq, sum(yc_spectra_samples[sample].values()))

In [ ]:
sems = {g:dict() for g in genotypes}
for g in genotypes:
    for context in yc_contexts:
        sems[g][context] = np.std([norm_yc_spectra_samples[s][context] for s in g_to_s(g)]) / (len(g_to_s(g)) ** (1/2))

fig, ax = plt.subplots(figsize=(len(vis_genotypes), 3))

c = gtools.light_cmap[9]
# yerrs = [[nyc_yerr_0[g]['YCNT'] for g in vis_genotypes], [nyc_yerr_1[g]['YCNT'] for g in vis_genotypes]]
yerrs = [sems[g]['YCNT'] for g in vis_genotypes]
bar_yc = ax.bar(range(0, len(vis_genotypes) * 3, 3), [norm_yc_spectra[genotype]['YCNT'] for genotype in vis_genotypes], yerr=yerrs, color=c, alpha=0.6)
ax.scatter([vis_genotypes.index(s_to_g(s)) * 3 + random.random() * 0.5 - 0.25 for s in vis_samples], [norm_yc_spectra_samples[s]['YCNT'] for s in vis_samples], color=c, s=[6 if len(g_to_s(s_to_g(s))) < 20 else 0.5 for s in vis_samples])

c = gtools.dark_cmap[9]
# yerrs = [[nyc_yerr_0[g]['RCNT'] for g in vis_genotypes], [nyc_yerr_1[g]['RCNT'] for g in vis_genotypes]]
yerrs = [sems[g]['RCNT'] for g in vis_genotypes]
bar_rc = ax.bar(range(1, len(vis_genotypes) * 3 + 1, 3), [norm_yc_spectra[genotype]['RCNT'] for genotype in vis_genotypes], yerr=yerrs, color=c, alpha=0.6)
ax.scatter([vis_genotypes.index(s_to_g(s)) * 3 + 1 + random.random() * 0.5 - 0.25 for s in vis_samples], [norm_yc_spectra_samples[s]['RCNT'] for s in vis_samples], color=c, s=[6 if len(g_to_s(s_to_g(s))) < 20 else 0.5 for s in vis_samples])
    
ax.set_xticks([x + 0.5 for x in range(0, len(vis_genotypes) * 3, 3)], labels=vis_genotypes)
ax.legend([bar_yc, bar_rc], ['Y(C>T)', 'R(C>T)'])
fig.suptitle('YC site vs RC site C>T rate', y=0.95)
fig.savefig('figs/yc_vs_rc_spectrum.svg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
for g in vis_genotypes:
    yc = [norm_yc_spectra_samples[s]['YCNT'] for s in g_to_s(g)]
    rc = [norm_yc_spectra_samples[s]['RCNT'] for s in g_to_s(g)]
    p = stats.ttest_ind(yc, rc).pvalue
    print(g, norm_yc_spectra[g]['YCNT'] / norm_yc_spectra[g]['RCNT'], p * len(vis_genotypes))

In [ ]:
col_diff = []
for s in g_to_s('untreated'):
    col_diff.append(norm_yc_spectra_samples[s]['YCNT'] / norm_yc_spectra_samples[s]['RCNT'])
stats.ttest_1samp(col_diff, norm_yc_spectra['MA lines']['YCNT'] / norm_yc_spectra['MA lines']['RCNT'])

## Indels

In [ ]:
indels = {g:{x:0 for x in range(-11, 12)} for g in genotypes}
for s in sample_names:
    for r in dfs_somatic[s].itertuples():
        if r.ref == '*':
            indels[s_to_g(s)][min(len(r.alt), 11)] += 1
        elif r.alt == '*':
            indels[s_to_g(s)][-1 * min(len(r.ref), 11)] += 1

In [ ]:
fig, axs = plt.subplots(len(vis_genotypes), figsize=(6, len(vis_genotypes) * 1), sharex=True)
for i, g in enumerate(vis_genotypes):
    axs[i].bar(indels[g].keys(), indels[g].values(), color=['tab:brown'] * 11 + ['tab:purple'] * 12, alpha=0.6)
    axs[i].set_title(g, x=0.01, y=0.5, ha='left')
axs[-1].set_xlabel('deletion (bp)                   insertion (bp)')
axs[-1].set_xticks([-11, -5, -1, 1, 5, 11], labels=['>10', '5', '1', '1', '5', '>10'])
axs[len(axs) // 2].set_ylabel('count')
fig.suptitle('indel sizes')
fig.savefig('figs/indel_sizes.svg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# make a correlation heatmap using all variant contexts
df_spectra = pd.DataFrame(norm_grouped_spectra)
df_spectra = df_spectra.transpose().rename(columns={i:genotypes[i] for i in range(len(genotypes))})
df_spectra['1001'] = list(norm_spectra[-1].values())


fig, axs = plt.subplots(2, figsize=(6, 9), gridspec_kw={'height_ratios': [1, 4], 'hspace':0})
Z = cluster.hierarchy.linkage(df_spectra.transpose(), method='ward', optimal_ordering=True, metric='euclidean')
dn = cluster.hierarchy.dendrogram(Z, ax=axs[0], no_labels=True)
df_spectra = df_spectra[[genotypes[i] if i < len(genotypes) else '1001' for i in dn['leaves']]]
df_corr = df_spectra.corr('pearson')
axs[1].imshow(df_corr)
axs[0].axis('off')
# axs[1].xaxis.tick_top()
axs[1].set_xticks(range(len(dn['leaves'])))
axs[1].set_xticklabels([genotypes[i] if i < len(genotypes) else '1001' for i in dn['leaves']], rotation=90)
axs[1].set_yticks(range(len(dn['leaves'])))
axs[1].set_yticklabels([genotypes[i] if i < len(genotypes) else '1001' for i in dn['leaves']])
fig.savefig('figs/mutation_spectra_heatmap.png', dpi=300, bbox_inches='tight')

In [ ]:
# find variant contexts with high variance for feature selection
variances = []
high_var_contexts = []
for c in contexts:
    rates = [spec[c] for spec in norm_grouped_spectra]
    var = np.var(rates)
    variances.append(var)
    if var > 0.00005:
        high_var_contexts.append(c)
        print(c, np.var(rates))

In [ ]:
plt.hist(variances, bins=[x / 100000 for x in range(100)], cumulative=1, histtype="step")
plt.show()
# 0.00005

In [ ]:
# check the variance of each context
# fig, axs = plt.subplots(len(genotypes) + 2, figsize=(14, len(genotypes)), sharex=True)
# colors = ['tab:blue'] * 16 + ['tab:orange'] * 16 + ['tab:green'] * 16 + ['tab:red'] * 16 + ['tab:purple'] * 16 + ['tab:cyan'] * 16 + ['tab:brown', 'tab:grey']
# for i in range(len(genotypes)):
#     axs[i].bar(range(len(contexts)), norm_grouped_spectra[i].values(), color=colors, alpha=0.6)
#     title = genotypes[i] + ' (n=' + str(grouped_counts[i]) + ')'
#     for j in range(len(sample_names)):
#         if sample_names[j].rsplit('-', 1)[0] == genotypes[i]:
#             axs[i].scatter([x + random.random() * 0.5 - 0.25 for x in range(len(contexts))], norm_spectra[j].values(), color=colors, s=1)
#     axs[i].set_title(title, y=0.55, x=0.01, loc='left')
#     axs[i].ticklabel_format(style='plain')
# axs[-2].bar(range(len(contexts)), norm_spectra[-1].values(), color=colors, alpha=0.6)
# axs[-2].set_title(f'1001 genomes (n={len(df_1001)})', y=0.55, x=0.01, loc='left')

# axs[-1].bar(range(len(contexts)), variances)

# axs[-1].ticklabel_format(style='plain')
# axs[-1].set_xticks(range(len(contexts)))
# axs[-1].set_xticklabels([c[0] + c[2] + c[-1] if c != 'ins' and c != 'del' else c for c in spectra[0].keys()], rotation=90, size=6)
# for pos, label in zip([7.5 + 16 * i for i in range(6)], ['A -> C', 'A -> G', 'A -> T', 'C -> A', 'C -> G', 'C -> T']): # trying to set separate major and minor ticks has a weird bug
#     axs[-1].text(pos, -0.001, label)
# axs[len(axs) // 2].set_ylabel('Proportion of mutations (context frequency normalized)')
# for ax in axs[:-1]:
#     ax.set_ylim(0, 0.15)
# fig.suptitle('Grouped spectra of somatic mutations', y=0.92)
# plt.show()

In [ ]:
# make the same correlation heatmap, but only use the high variance contexts
df_spectra = pd.DataFrame([{context:rate for context, rate in spec.items() if context in high_var_contexts} for spec in norm_grouped_spectra])
df_spectra = df_spectra.transpose().rename(columns={i:genotypes[i] for i in range(len(genotypes))})
df_spectra['1001'] = [rate for context, rate in norm_spectra[-1].items() if context in high_var_contexts]


fig, axs = plt.subplots(2, figsize=(6, 9), gridspec_kw={'height_ratios': [1, 4], 'hspace':0})
Z = cluster.hierarchy.linkage(df_spectra.transpose(), 'ward', optimal_ordering=True)
dn = cluster.hierarchy.dendrogram(Z, ax=axs[0], no_labels=True)
df_spectra = df_spectra[[genotypes[i] if i < len(genotypes) else '1001' for i in dn['leaves']]]
df_corr = df_spectra.corr('pearson')
axs[1].imshow(df_corr)
axs[0].axis('off')
# axs[1].xaxis.tick_top()
axs[1].set_xticks(range(len(dn['leaves'])))
axs[1].set_xticklabels([genotypes[i] if i < len(genotypes) else '1001' for i in dn['leaves']], rotation=90)
axs[1].set_yticks(range(len(dn['leaves'])))
axs[1].set_yticklabels([genotypes[i] if i < len(genotypes) else '1001' for i in dn['leaves']])
fig.savefig('figs/mutation_spectra_heatmap.png', dpi=300, bbox_inches='tight')

In [ ]:
# avg_bqs = df_tmp.bq.apply(lambda x: sum([ord(b) - 33 for b in x]) / len(x))
# fig, ax = plt.subplots(figsize=(6, 3))
# ax.hist(avg_bqs)

In [ ]:
# fig = plt.figure(constrained_layout=True, figsize=(6, 6))
# gs = fig.add_gridspec(2, 2, width_ratios=(4, 1), height_ratios=(1, 4))
# ax_s = fig.add_subplot(gs[1,0])
# ax_s.scatter(avg_bqs, var_pos, alpha=0.1)
# fig.add_subplot(gs[0,0], sharex=ax_s).hist(avg_bqs, bins=range(42))
# fig.add_subplot(gs[1,1], sharey=ax_s).hist(var_pos, orientation='horizontal', bins=range(151))
# fig.suptitle('Duplex seq base quality and position in read of putative mutations')
# ax_s.set_xlabel('Base quality')
# ax_s.set_ylabel('Position in read')
# fig.savefig('figs/duplex_seq_base_quality_and_position_of_variants.png', dpi=300, bbox_inches='tight')

In [ ]:
# control_bq_hist = np.zeros(45, dtype='i')
# control_mq_hist = np.zeros(45, dtype='i')

# aln = pysam.AlignmentFile('../data/align/uvtest_UVC_C1_filtered.bam', 'rb')

# for i, read in tqdm(enumerate(aln.fetch())):
#     if i > 100000:
#         break
#     if not read.is_proper_pair:
#         continue
    
#     for bq in read.query_qualities:
#         control_bq_hist[bq] += 1
    
#     if not read.get_tag('MD').isdigit():
#         control_mq_hist[read.mapping_quality] += 1
    

In [ ]:
# mut_bq_hist = np.zeros(45, dtype='i')
# mut_mq_hist = np.zeros(45, dtype='i')

# for r in df_all.itertuples():
#     for bq in r.bq:
#         mut_bq_hist[ord(bq) - 33] += 1
#     for mq in r.mq:
#         mut_mq_hist[ord(mq) - 33] += 1

In [ ]:
# fig, axs = plt.subplots(2, 2, sharey=True, sharex=True)
# axs[0][0].bar(range(len(control_bq_hist)), control_bq_hist / sum(control_bq_hist))
# axs[1][0].bar(range(len(mut_bq_hist)), mut_bq_hist / sum(mut_bq_hist))
# axs[0][1].bar(range(len(control_mq_hist)), control_mq_hist / sum(control_mq_hist))
# axs[1][1].bar(range(len(mut_mq_hist)), mut_mq_hist / sum(mut_mq_hist))
# axs[0][0].set_title('BQ')
# axs[0][0].set_ylabel('Null distribution')
# axs[0][1].set_title('MQ')
# axs[1][0].set_ylabel('Distribution at mutations')
# fig.suptitle('BQ and MQ distributions')
# fig.savefig('figs/mq_and_bq_of_mutated_reads.png', dpi=300, bbox_inches='tight')

# Selection

In [ ]:
gff_cols = 'chrom source kind start end score strand phase att'.split()
df_gff = pd.read_table('data/ref/ref.gff', names=gff_cols) # can get the gff from https://www.arabidopsis.org/api/download-files/download?filePath=Genes/TAIR10_genome_release/TAIR10_gff3/TAIR10_GFF3_genes_transposons.gff
df_gff.start -= 1 # convert gff to 0-base half open

In [ ]:
genome = gtools.load_genome(genome_fasta)

In [ ]:
rep_gene_models = set()
with open('data/ref/TAIR10_representative_gene_models.txt', 'r') as f: # can get this from https://www.arabidopsis.org/api/download-files/download?filePath=Genes/TAIR10_genome_release/TAIR10_gene_lists/TAIR10_representative_gene_models
    for l in f:
        if l[:2] != 'AT':
            continue
        rep_gene_models.add(l.strip())
print(len(rep_gene_models))

# there are also representative gene models for transposable elements and pseudogenes, so restrict the representative models to only protein coding genes
protein_genes = set([x.split("ID=")[1].split(";")[0] for x in df_gff[df_gff.kind == "gene"].att])
rep_gene_models = {x for x in rep_gene_models if x.split('.')[0] in protein_genes}
assert len(protein_genes) == len(rep_gene_models)
len(protein_genes), len(rep_gene_models)

In [ ]:
# reverse complement a sequence
comp = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
def rc(seq):
    ret = ''
    for nuc in reversed(seq):
        ret += comp[nuc]
    return ret

In [ ]:
# identify all synonymous and nonsynonymous sites in the genome
s_sites = {chrom:np.zeros(chrom_sizes[chrom], dtype=bool) for chrom in chrom_sizes}
ns_sites = {chrom:np.zeros(chrom_sizes[chrom], dtype=bool) for chrom in chrom_sizes}

df_cds = df_gff[(df_gff.kind == 'CDS')]
for idx, row in tqdm(enumerate(df_cds.itertuples()), total=len(df_cds)):
    if row.chrom not in chrom_sizes:
        continue
    
    # only consider the representative gene model (transcript)
    trans = row.att.split('Parent=')[1].split(',')[0]
    if trans not in rep_gene_models:
        continue
    
    
    if row.strand == '+':
        for i, pos in enumerate(range(row.start, row.end)):
            syn = False
            codon_phase = (i - int(row.phase)) % 3
            if codon_phase == 2: # if the third base of a codon
                if i == 0: # if the first base of the cds element (exon)
                    prev_end = df_cds.iloc[idx - 1].end
                    codon = genome[row.chrom][prev_end - 2:prev_end]
                elif i == 1: # if the second base of the cds element (exon)
                    prev_end = df_cds.iloc[idx - 1].end
                    codon = genome[row.chrom][prev_end - 1] + genome[row.chrom][pos - 1]
                else:
                    codon = genome[row.chrom][pos - 2:pos]
                
                if codon in {'CT', 'GT', 'TC', 'CC', 'AC', 'GC', 'CG', 'GG'}:
                    syn = True
            
            if syn:
                s_sites[row.chrom][pos] = 1
            else:
                ns_sites[row.chrom][pos] = 1
    else: # do the same thing for the reverse strand
        for i, pos in enumerate(range(row.end - 1, row.start - 1, -1)):
            syn = False
            codon_phase = (i - int(row.phase)) % 3
            if codon_phase == 2:
                if i == 0:
                    next_start = df_cds.iloc[idx + 1].start
                    codon = rc(genome[row.chrom][next_start:next_start + 2])
                elif i == 1:
                    next_start = df_cds.iloc[idx + 1].start
                    codon = rc(genome[row.chrom][pos + 1] + genome[row.chrom][next_start])
                else:
                    codon = rc(genome[row.chrom][pos + 1:pos + 3])
                
                if codon in {'CT', 'GT', 'TC', 'CC', 'AC', 'GC', 'CG', 'GG'}:
                    syn = True
            
            if syn:
                s_sites[row.chrom][pos] = 1
            else:
                ns_sites[row.chrom][pos] = 1

In [ ]:
# # make bed files to visually confirm sites are getting the correct label
# ns_f = open('tmp/ns_sites.bed', 'w')
# s_f = open('tmp/s_sites.bed', 'w')
# for i in range(100000):
#     if ns_sites['Chr1'][i] == 1:
#         ns_f.write(f'Chr1\t{str(i)}\t{str(i + 1)}\n')
#     if s_sites['Chr1'][i] == 1:
#         s_f.write(f'Chr1\t{str(i)}\t{str(i + 1)}\n')
# ns_f.close()
# s_f.close()

In [ ]:
# get the mutation count and coverage of synonymous and nonsynonymous sites for each sample
dnds = {s: {'s_count':0, 'ns_count':0, 'other_count':0, 's_cov':0, 'ns_cov':0} for s in sample_names}
for s in tqdm(sample_names):
    # load the callable coverage for the sample
    call_cov = dict()
    for chrom in chrom_sizes:
        if 'MA lines' or '1001' in s:
            call_cov[chrom] = np.load(f'{cov_prefixes["untreated_1"]}{chrom}.npy') > 0
        elif 'Ler-0' in s:
            call_cov[chrom] = np.load(f'{cov_prefixes[sample]}{chrom}.npy') > 0
        else:
            call_cov[chrom] = np.load(f'{cov_prefixes[sample]}{chrom}.npy')
    
    
    # count the mutations at synonymous and nonsynonymous sites
    s_count = 0
    ns_count = 0
    other_count = 0
    for row in dfs_somatic[s].itertuples():
        if s_sites[row.chrom][row.pos]:
            s_count += 1
        elif ns_sites[row.chrom][row.pos]:
            ns_count += 1
        else:
            other_count += 1
    
    # count the coverage at synonymous and nonsynonymous sites
    s_cov = 0
    ns_cov = 0
    for chrom in chrom_sizes:
        s_cov += np.sum(s_sites[row.chrom] * call_cov[row.chrom])
        ns_cov += np.sum(ns_sites[row.chrom] * call_cov[row.chrom])
    
    # add values to dnds
    dnds[s]['s_count'] = s_count
    dnds[s]['ns_count'] = ns_count
    dnds[s]['other_count'] = other_count
    dnds[s]['s_cov'] = s_cov
    dnds[s]['ns_cov'] = ns_cov
    
    # add values to 'all somatic'
    # if 'Ler-0-germline' not in s and '1001' not in s and 'MA lines' not in s:
    #     dnds['all somatic']['s_count'] += s_count
    #     dnds['all somatic']['ns_count'] += ns_count
    #     dnds['all somatic']['s_cov'] += s_cov
    #     dnds['all somatic']['ns_cov'] += ns_cov


In [ ]:
# add a rate ratio to dnds
cannot_compute = []
for s in dnds:
    if dnds[s]['s_count'] == 0:
        cannot_compute.append(s)
        dnds[s]['ratio'] = np.nan
        continue
    dnds[s]['ratio'] = (dnds[s]['ns_count'] / dnds[s]['ns_cov']) / (dnds[s]['s_count'] / dnds[s]['s_cov'])
print(f'could not compute dnds for {cannot_compute}, no synonymous mutations')

In [ ]:
lower = dict()
upper = dict()
for g in dnds:
    boots = []
    reps = 10000 if dnds[g]['ns_count'] < 10000 else 20 # number of bootstrap replicates, use a small number for the genotypes with a ton of mutations
    for i in tqdm(range(reps)):
        # randomly sample the same number of mutations with replacement
        total_count = dnds[g]['s_count'] + dnds[g]['ns_count'] + dnds[g]['other_count']
        choices = random.choices([0, 1, 2], weights=[dnds[g]['s_count'] / total_count, dnds[g]['ns_count'] / total_count, dnds[g]['other_count'] / total_count], k=total_count)
        
        # same as above, but don't consider the mutations in non-CDS regions; I tried both ways and didn't see any difference
        # total_count = dnds[g]['s_count'] + dnds[g]['ns_count']
        # choices = random.choices([0, 1], weights=[dnds[g]['s_count'] / total_count, dnds[g]['ns_count'] / total_count], k=total_count)
        
        # calculate the dnds of this sample
        boot_s = choices.count(0)
        boot_ns = choices.count(1)
        if boot_s > 0:
            boots.append((boot_ns / dnds[g]['ns_cov']) / (boot_s / dnds[g]['s_cov']))
        else:
            boots.append(2)
    
    # pick the 5th and 95th percentile dnds as the lower and upper confidence interval
    boots.sort()
    lower[g] = boots[int(len(boots) * 0.05)]
    upper[g] = boots[int(len(boots) * 0.95)]


In [ ]:
all_somatic = ['untreated', 'UVB', 'UVC']
vis_genotypes = ['MA lines', 'Ler-0', '1001']

fig, ax = plt.subplots(figsize=(4, 6))
ax.hlines(1, -1, len(dnds) + 1, color='black')

sems = dict()
a = []
for g in all_somatic + vis_genotypes:
    ratios = [dnds[s]['ratio'] for s in g_to_s(g)]
    if np.nan not in ratios:
        sems[g] = np.std(ratios) / (len(ratios) ** (1/2))
    else:
        print(f'WARNING: undefined dnds values for some samples in {g}, bootstrapping the sem')
        boots = []
        for b in range(2000):
            chosen = random.choices(g_to_s(g), k=len(g_to_s(g)))
            ns_count = sum([dnds[s]['ns_count'] for s in chosen])
            ns_cov = sum([dnds[s]['ns_cov'] for s in chosen])
            s_count = sum([dnds[s]['s_count'] for s in chosen])
            s_cov = sum([dnds[s]['s_cov'] for s in chosen])
            ratio = (ns_count / ns_cov) / (s_count / s_cov)
            boots.append(ratio)
        sems[g] = np.std(boots)
        
    if g in all_somatic:
        a += ratios
sems['all somatic'] = np.std(a) / (len(a) ** (1/2))

ns_count, ns_cov, s_count, s_cov = 0, 0, 0, 0
for g in all_somatic:
    ns_count += sum([dnds[s]['ns_count'] for s in g_to_s(g)])
    ns_cov += sum([dnds[s]['ns_cov'] for s in g_to_s(g)])
    s_count += sum([dnds[s]['s_count'] for s in g_to_s(g)])
    s_cov += sum([dnds[s]['s_cov'] for s in g_to_s(g)])
ratio = (ns_count / ns_cov) / (s_count / s_cov)
ax.bar(0, ratio, yerr=sems['all somatic'], color='tab:grey')

for g in all_somatic:
    for s in g_to_s(g):
        ratio = (dnds[s]['ns_count'] / dnds[s]['ns_cov']) / (dnds[s]['s_count'] / dnds[s]['s_cov'])
        ax.scatter(-0.25 + random.random() * 0.5, ratio, color=gen_cmap(g))

for i, g in enumerate(vis_genotypes):
    print(g)
    ns_count = sum([dnds[s]['ns_count'] for s in g_to_s(g)])
    ns_cov = sum([dnds[s]['ns_cov'] for s in g_to_s(g)])
    s_count = sum([dnds[s]['s_count'] for s in g_to_s(g)])
    s_cov = sum([dnds[s]['s_cov'] for s in g_to_s(g)])
    ratio = (ns_count / ns_cov) / (s_count / s_cov)
    print(g, ns_count, ns_cov, s_count, s_cov)
    ax.bar(i + 1, ratio, yerr=sems[g], color=gen_cmap(g))

ax.set_xlim(-0.5, len(vis_genotypes) + 0.5)
# ax.set_ylim(0, 2)
ax.set_xticks(range(len(vis_genotypes) + 1))
ax.set_xticklabels(['all somatic'] + vis_genotypes, rotation=45, ha='right')
ax.set_title('dN/dS ratios by sample')
ax.set_ylabel('dN/dS')
fig.savefig('figs/dnds_across_samples.svg', dpi=300, bbox_inches='tight')
plt.show()

# Look for stranded results

In [ ]:
f_gene = {chrom:np.zeros(chrom_sizes[chrom], dtype=bool) for chrom in chrom_sizes}
r_gene = {chrom:np.zeros(chrom_sizes[chrom], dtype=bool) for chrom in chrom_sizes}

for row in df_gff[df_gff.kind == 'mRNA'].itertuples():
    if row.att.split('ID=')[1].split(';')[0] not in rep_gene_models or row.chrom in ['ChrC', 'ChrM']:
        continue
    
    if row.strand == '+':
        f_gene[row.chrom][row.start:row.end] = True
    else:
        r_gene[row.chrom][row.start:row.end] = True

In [ ]:
simple_contexts = ['CA', 'CG', 'CT', 'TA', 'TC', 'TG', 'del', 'ins']
pyr_forward_s = {s:{c:0 for c in simple_contexts} for s in sample_names}
pyr_reverse_s = {s:{c:0 for c in simple_contexts} for s in sample_names}
pyr_forward_g = {g:{c:0 for c in simple_contexts} for g in genotypes}
pyr_reverse_g = {g:{c:0 for c in simple_contexts} for g in genotypes}

for s in tqdm(sample_names):
    for row in dfs_somatic[s].itertuples():
        if f_gene[row.chrom][row.pos] and not r_gene[row.chrom][row.pos]:
            reverse = False
        elif r_gene[row.chrom][row.pos] and not f_gene[row.chrom][row.pos]:
            reverse = True
        else:
            # if s == 'Col-0-8':
            #     print(f'{row.chrom}:{row.pos} {row.ref}>{row.alt} not in gene')
            continue
        
        if row.ref == '*':
            context = 'ins'
        elif row.alt == '*':
            context = 'del'
        elif row.ref == 'C' or row.ref == 'T':
            context = row.ref + row.alt
        elif row.ref == 'A' or row.ref == 'G':
            context = comp[row.ref] + comp[row.alt]
            reverse = not reverse
        
        g = s_to_g(s)
        if reverse:
            pyr_reverse_s[s][context] += 1
            pyr_reverse_g[g][context] += 1
        else:
            pyr_forward_s[s][context] += 1
            pyr_forward_g[g][context] += 1
            

In [ ]:
vis_genotypes = genotypes
vis_samples = [s for s in sample_names if s_to_g(s) in ['Col-0', 'UVB', 'UVC']]

sems = dict()
for g in genotypes:
    try:
        sems[g] = np.std([pyr_forward_s[s]['CT'] / pyr_reverse_s[s]['CT'] for s in g_to_s(g)]) / (len(g_to_s(g)) ** (1/2))
    except ZeroDivisionError:
        print(f'WARNING: some samples in {g} have undefined ratios, calculating a bootstrap sem')
        try:
            boot_means = []
            for b in range(2000):
                f = sum(random.choices([pyr_forward_s[s]['CT'] for s in g_to_s(g)], k=len(g_to_s(g))))
                r = sum(random.choices([pyr_reverse_s[s]['CT'] for s in g_to_s(g)], k=len(g_to_s(g))))
                boot_means.append(f / r)
            sems[g] = np.std(boot_means)
        except ZeroDivisionError:
            print(f'WARNING: some bootstrpas in {g} also have undefinded ratios, setting sem to 0')
            sems[g] = 0

fig, ax = plt.subplots(figsize=(3.5, 4))
for i, g in enumerate(vis_genotypes):
    forward = pyr_forward_g[g]['CT']
    reverse = pyr_reverse_g[g]['CT']
    forward_cinf = stats.binom.interval(0.95, forward + reverse, forward / (forward + reverse))
    ratio = forward / reverse
    low_err = ratio - (forward_cinf[0] / (forward + reverse - forward_cinf[0]))
    high_err = (forward_cinf[1] / (forward + reverse - forward_cinf[1])) - ratio
    ax.bar(i, ratio, yerr=sems[g], color=gen_cmap(g), alpha=0.8)
for s in vis_samples:
    ratio = pyr_forward_s[s]['CT'] / pyr_reverse_s[s]['CT']
    ax.scatter(vis_genotypes.index(s_to_g(s)) + random.random() * 0.5 - 0.25, ratio, color=gen_cmap(s_to_g(s)))
ax.hlines(1, -10, 10, color='black')
ax.set_xlim(-0.75, len(vis_genotypes) - 0.25)
ax.set_xticks(range(len(vis_genotypes)), vis_genotypes)
ax.set_ylabel('nontemplate / template C>T mutations')
fig.savefig('figs/strand_bias_of_ct_mutations.svg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
for g in vis_genotypes:
    try:
        print(g, stats.ttest_1samp([pyr_forward_s[s]['CT'] / pyr_reverse_s[s]['CT'] for s in g_to_s(g)], 1, alternative='greater').pvalue * 4)
    except ZeroDivisionError:
        print(g, 'error')
        continue

# Mutation rate by genomic region

In [ ]:
regions = { # these files are generated by bed_regions.ipynb
    'exon':'data/region/exon.bed',
    'intron':'data/region/intron.bed',
    'te':'data/region/te.bed',
    'promoter':'data/region/promoter.bed',
    'intergenic':'data/region/intergenic.bed',
    'acr':'data/region/acrs.bed',
    'gene':'data/region/protein_mRNA.bed',
    'pericentromere':'data/region/pericentromere.bed',
    'met cg':'data/region/met_cg.bed',
    'met chg':'data/region/met_chg.bed',
    'met chh':'data/region/met_chh.bed',
    'unmet cg':'data/region/unmet_cg.bed',
    'unmet chg':'data/region/unmet_chg.bed',
    'unmet chh':'data/region/unmet_chh.bed',
    # 'near met':'data/region/near_met.bed',
}

region_arrays = {r:{chrom:np.zeros(chrom_sizes[chrom], dtype=bool) for chrom in chrom_sizes} for r in regions}
for r in tqdm(region_arrays):
    df_bed = pd.read_table(regions[r], names='chrom start end'.split(), usecols=list(range(3)))
    df_bed = df_bed[df_bed.chrom.apply(lambda c: c in chrom_sizes)]
    for row in df_bed.itertuples():
        region_arrays[r][row.chrom][row.start:row.end] = True
    del df_bed

In [ ]:
# add extra region arrays using the existing ones

region_arrays['pericentromeric genes'] = {chrom:region_arrays['gene'][chrom] * region_arrays['pericentromere'][chrom] for chrom in chrom_sizes}
region_arrays['distal genes'] = {chrom:region_arrays['gene'][chrom] * ~region_arrays['pericentromere'][chrom] for chrom in chrom_sizes}
region_arrays['pericentromeric tes'] = {chrom:region_arrays['te'][chrom] * region_arrays['pericentromere'][chrom] for chrom in chrom_sizes}
region_arrays['distal tes'] = {chrom:region_arrays['te'][chrom] * ~region_arrays['pericentromere'][chrom] for chrom in chrom_sizes}

region_arrays['genic met cg'] = {chrom:region_arrays['met cg'][chrom] * region_arrays['gene'][chrom] for chrom in chrom_sizes}
region_arrays['genic unmet cg'] = {chrom:region_arrays['unmet cg'][chrom] * region_arrays['gene'][chrom] for chrom in chrom_sizes}
region_arrays['nongenic met cg'] = {chrom:region_arrays['met cg'][chrom] * ~region_arrays['gene'][chrom] for chrom in chrom_sizes}
region_arrays['nongenic unmet cg'] = {chrom:region_arrays['unmet cg'][chrom] * ~region_arrays['gene'][chrom] for chrom in chrom_sizes}
region_arrays['nongenic met chg'] = {chrom:region_arrays['met chg'][chrom] * ~region_arrays['gene'][chrom] for chrom in chrom_sizes}
region_arrays['nongenic unmet chg'] = {chrom:region_arrays['unmet chg'][chrom] * ~region_arrays['gene'][chrom] for chrom in chrom_sizes}
region_arrays['nongenic met chh'] = {chrom:region_arrays['met chh'][chrom] * ~region_arrays['gene'][chrom] for chrom in chrom_sizes}
region_arrays['nongenic unmet chh'] = {chrom:region_arrays['unmet chh'][chrom] * ~region_arrays['gene'][chrom] for chrom in chrom_sizes}

region_arrays['genic acr'] = {chrom:region_arrays['acr'][chrom] * region_arrays['gene'][chrom] for chrom in chrom_sizes}
region_arrays['genic nonacr'] = {chrom:~region_arrays['acr'][chrom] * region_arrays['gene'][chrom] for chrom in chrom_sizes}
region_arrays['nongenic acr'] = {chrom:region_arrays['acr'][chrom] * ~region_arrays['gene'][chrom] for chrom in chrom_sizes}
region_arrays['nongenic nonacr'] = {chrom:~region_arrays['acr'][chrom] * ~region_arrays['gene'][chrom] for chrom in chrom_sizes}

# region_arrays['exon nomet'] = {chrom:region_arrays['exon'][chrom] * (region_arrays['at'][chrom] + region_arrays['unmet cg'][chrom] + region_arrays['unmet chg'][chrom] + region_arrays['unmet chh'][chrom]) for chrom in chrom_sizes}
# region_arrays['intron nomet'] = {chrom:region_arrays['intron'][chrom] * (region_arrays['at'][chrom] + region_arrays['unmet cg'][chrom] + region_arrays['unmet chg'][chrom] + region_arrays['unmet chh'][chrom]) for chrom in chrom_sizes}
# region_arrays['te nomet'] = {chrom:region_arrays['te'][chrom] * (region_arrays['at'][chrom] + region_arrays['unmet cg'][chrom] + region_arrays['unmet chg'][chrom] + region_arrays['unmet chh'][chrom]) for chrom in chrom_sizes}
# region_arrays['promoter nomet'] = {chrom:region_arrays['promoter'][chrom] * (region_arrays['at'][chrom] + region_arrays['unmet cg'][chrom] + region_arrays['unmet chg'][chrom] + region_arrays['unmet chh'][chrom]) for chrom in chrom_sizes}
# region_arrays['intergenic nomet'] = {chrom:region_arrays['intergenic'][chrom] * (region_arrays['at'][chrom] + region_arrays['unmet cg'][chrom] + region_arrays['unmet chg'][chrom] + region_arrays['unmet chh'][chrom]) for chrom in chrom_sizes}

# region_arrays['exon yc'] = {chrom:region_arrays['exon'][chrom] * yc_mask[chrom] for chrom in chrom_sizes}
# region_arrays['intron yc'] = {chrom:region_arrays['intron'][chrom] * yc_mask[chrom] for chrom in chrom_sizes}
# region_arrays['te yc'] = {chrom:region_arrays['te'][chrom] * yc_mask[chrom] for chrom in chrom_sizes}
# region_arrays['promoter yc'] = {chrom:region_arrays['promoter'][chrom] * yc_mask[chrom] for chrom in chrom_sizes}
# region_arrays['intergenic yc'] = {chrom:region_arrays['intergenic'][chrom] * yc_mask[chrom] for chrom in chrom_sizes}


# region_arrays['cg'] = {chrom:np.array([n == 'C' or n == 'G' for n in genome[chrom]]) for chrom in chrom_sizes}
# region_arrays['at'] = {chrom:np.array([n == 'A' or n == 'T' for n in genome[chrom]]) for chrom in chrom_sizes}

# for region in 'exon intron te promoter intergenic'.split():
    # region_arrays[f'{region} cg'] = {chrom:region_arrays[region][chrom] * region_arrays['cg'][chrom] for chrom in chrom_sizes}
    # region_arrays[f'{region} at'] = {chrom:region_arrays[region][chrom] * region_arrays['at'][chrom] for chrom in chrom_sizes}
    # region_arrays[f'{region} pericentromere'] = {chrom:region_arrays[region][chrom] * region_arrays['pericentromere'][chrom] for chrom in chrom_sizes}
    # region_arrays[f'{region} distal'] = {chrom:region_arrays[region][chrom] * ~region_arrays['pericentromere'][chrom] for chrom in chrom_sizes}
    # region_arrays[f'{region} met'] = {chrom:region_arrays[region][chrom] * region_arrays['met c'][chrom] for chrom in chrom_sizes}
    # region_arrays[f'{region} unmet'] = {chrom:region_arrays[region][chrom] * region_arrays['unmet c'][chrom] for chrom in chrom_sizes}
    # region_arrays[f'{region} acr'] = {chrom:region_arrays[region][chrom] * region_arrays['acr'][chrom] for chrom in chrom_sizes}
    # region_arrays[f'{region} nonacr'] = {chrom:region_arrays[region][chrom] * ~region_arrays['acr'][chrom] for chrom in chrom_sizes}
    

In [ ]:
del region_arrays['met cg'], region_arrays['met chg'], region_arrays['met chh'], region_arrays['unmet cg'], region_arrays['unmet chg'], region_arrays['unmet chh']


In [ ]:
def array_to_bed(array, bed):
    fout = open(bed, 'w')
    for chrom in chrom_sizes:
        region_start = None
        for i, value in enumerate(array[chrom]):
            if value and region_start is None:
                region_start = i
            elif value:
                pass
            elif region_start is None:
                pass
            else:
                fout.write(f'{chrom}\t{region_start}\t{i}\n')
                region_start = None
    fout.close()

In [ ]:
# array_to_bed(region_arrays['intron acr'], 'tmp/region_intron_acr.bed')
# array_to_bed(region_arrays['intron nonacr'], 'tmp/region_intron_nonacr.bed')

In [ ]:
# count the number of mutations in each element type for each sample
elem_counts = { # indexed as elem_counts[thing][sample/genotype index][element type] = mutations in element
    'ind':{s:{r:0 for r in region_arrays} for s in dfs_somatic}, # individual samples
    'gen':{g:{r:0 for r in region_arrays} for g in genotypes}, # sum over all samples with the same genotype
    'low':{g:{r:0 for r in region_arrays} for g in genotypes}, # lower confidence interval by genotype
    'high':{g:{r:0 for r in region_arrays} for g in genotypes}, # upper confidence interval by genotype
    'gen_uniform':{g:0 for g in genotypes}
}
for sample, df in tqdm(dfs_somatic.items()):
    genotype = s_to_g(sample)
    for row in df.itertuples():
        for r in region_arrays:
            if region_arrays[r][row.chrom][row.pos]:
                c = 1
                # if row.ref == 'C' or row.ref == 'G': # adjust for different rates in CG vs AT sites
                #     c = 1 / cg_at_ratio[genotype]
                elem_counts['ind'][sample][r] += c
                elem_counts['gen'][genotype][r] += c
    # elem_counts['gen_uniform'][genotype] += sum((df.ref != 'C') & (df.ref != 'G')) + sum((df.ref == 'C') | (df.ref == 'G')) / cg_at_ratio[genotype]
    elem_counts['gen_uniform'][genotype] += len(df)

In [ ]:
# generate 95% confidence intervals using binomial distributions
for g in tqdm(genotypes):
    for t in region_arrays:
        muts_in_elem = elem_counts['gen'][g][t]
        total_muts = elem_counts['gen_uniform'][g]
        interval = stats.binom.interval(0.95, total_muts, muts_in_elem / total_muts)
    
        elem_counts['low'][g][t] = interval[0]
        elem_counts['high'][g][t] = interval[1]

In [ ]:
# generate chi-squared p-values that each genotype differs from Col-0. 
# note: these don't really have much meaning
p_vals = []
for g in genotypes:
    assert elem_counts['gen'][g].keys() == region_arrays.keys()
    act = list(elem_counts['gen'][g].values())
    exp = list(elem_counts['gen'][genotypes[0]].values())
    xs = stats.chisquare(act, [x * (sum(act) / sum(exp)) for x in exp])
    p_vals.append(xs.pvalue)

In [ ]:
elem_cov = {
    'ind':{s:{t:0 for t in region_arrays} for s in sample_names}, # individual samples
    'gen':{g:{t:0 for t in region_arrays} for g in genotypes}, # sum over all samples with the same genotype
    'uniform':{g:0 for g in genotypes}
}

wgs_cov_arrs = {chrom:np.load(f'{cov_prefixes["untreated_1"]}{chrom}.npy') > 0 for chrom in chrom_sizes}
wgs_cov_sum = sum([np.sum(wgs_cov_arrs[chrom]) for chrom in chrom_sizes])
# first_ma_sample = [x for x in sample_names if 'MA lines' in x or '1001' in x][0] if 'MA lines' in genotypes else None
for s in tqdm(sample_names):
    g = s_to_g(s)
    
    # if 'MA lines' in s and s != first_ma_sample: # skip recalculating MA line coverage since they're all the same FIXME this code changed the results once, but it was probably because non-zero conflict rates were entered for the MA lines
    #     for t in region_arrays:
    #         first_cov = elem_cov['ind'][first_ma_sample][t]
    #         elem_cov['ind'][s][t] = first_cov
    #         elem_cov['gen'][g][t] += first_cov
    #     elem_cov['uniform'][g] += wgs_cov_sum
    #     continue
    
    for chrom in chrom_sizes:
        if 'Ler-0' in s:
            cov_arr = np.load(f'{cov_prefixes[s]}{chrom}.npy')
            cov_arr = cov_arr > 0
        elif 'MA lines' in s or '1001' in s:
            cov_arr = wgs_cov_arrs[chrom]
        else:
            cov_arr = np.load(f'{cov_prefixes[s]}{chrom}.npy')
        for t in region_arrays:
            c = np.sum(region_arrays[t][chrom] * cov_arr) * (1 - conflict_rates[sample_names.index(s)])
            elem_cov['ind'][s][t] += c
            elem_cov['gen'][g][t] += c
        elem_cov['uniform'][g] += np.sum(cov_arr) * (1 - conflict_rates[sample_names.index(s)])
del wgs_cov_arrs

In [ ]:
# convert the proportion of mutations in each element to a rate using the callable coverage
elem_rates = { # indexed as elem_counts[thing][sample/genotype index][element type] = mutation rate in element (mutations / callable bp in first sample)
    'ind':{s:{t:0 for t in region_arrays} for s in sample_names}, # individual samples
    'gen':{g:{t:0 for t in region_arrays} for g in genotypes}, # sum over all samples with the same genotype
    'low':{g:{t:0 for t in region_arrays} for g in genotypes}, # lower confidence interval by genotype
    'high':{g:{t:0 for t in region_arrays} for g in genotypes}, # upper confidence interval by genotype
    'gen_uniform': {g:0 for g in genotypes}
}

for thing in elem_counts:
    for sg in elem_counts[thing]: # sg is either the sample or the genotype
        if thing != 'gen_uniform':
            cov_thing = 'ind' if thing == 'ind' else 'gen'
            for t in region_arrays:
                elem_rates[thing][sg][t] = elem_counts[thing][sg][t] / elem_cov[cov_thing][sg][t]
        else:
            elem_rates[thing][sg] = elem_counts[thing][sg] / elem_cov['uniform'][sg]

In [ ]:
# normalize the mutation rates in each element to the uniform rate
norm_elem_rates = { # indexed as elem_counts[thing][sample/genotype index][element type] = mutation rate in element / genome-wide rate
    'ind':{s:{t:0 for t in region_arrays} for s in sample_names}, # individual samples
    'gen':{g:{t:0 for t in region_arrays} for g in genotypes}, # sum over all samples with the same genotype
    'low':{g:{t:0 for t in region_arrays} for g in genotypes}, # lower confidence interval by genotype
    'high':{g:{t:0 for t in region_arrays} for g in genotypes}, # upper confidence interval by genotype
    'gen_uniform': {g:0 for g in genotypes}
}

for thing in elem_counts:
    for sg in elem_rates[thing]:
        if thing == 'ind': # if looking at an individual sample, normalize by the genotype's uniform rate
            g = s_to_g(sg)
            # print(f'{sg} h3k9 rate = {elem_rates[thing][sg][t]}, uniform rate = {elem_rates["gen_uniform"][g]}')
            norm_elem_rates[thing][sg] = {t:elem_rates[thing][sg][t] / elem_rates['gen_uniform'][g] for t in region_arrays}
        elif thing != 'gen_uniform': # normalize the uniform rate by itself, should be 1
            norm_elem_rates[thing][sg] = {t:elem_rates[thing][sg][t] / elem_rates['gen_uniform'][sg] for t in region_arrays}
        else:
            norm_elem_rates[thing][sg] = elem_rates[thing][sg] / elem_rates['gen_uniform'][sg]

In [ ]:
vis_genotypes = genotypes
unique_samples = [s for s in sample_names if [s_to_g(x) for x in sample_names].count(s_to_g(s)) == 1]
vis_samples = [s for s in sample_names if s_to_g(s) in vis_genotypes and s not in unique_samples and s_to_g(s) != 'MA lines']
vis_regions = ['exon', 'intron', 'te', 'promoter', 'intergenic']
# vis_regions = ['pericentromeric genes', 'distal genes', 'pericentromeric tes', 'distal tes']
# vis_regions = ['pericentromeric syn', 'distal syn', 'pericentromeric nonsyn', 'distal nonsyn']
# vis_regions = ['genic unmet cg', 'genic met cg', 'nongenic unmet cg', 'nongenic met cg', 'nongenic unmet chg', 'nongenic met chg', 'nongenic unmet chh', 'nongenic met chh']
# vis_regions = ['genic acr', 'genic nonacr', 'nongenic acr', 'nongenic nonacr']#, 'nongenic acr unmet distal', 'nongenic nonacr unmet distal']
# vis_regions = ['exon acr', 'exon nonacr', 'intron acr', 'intron nonacr', 'te acr', 'te nonacr', 'promoter acr', 'promoter nonacr', 'intergenic acr', 'intergenic nonacr']
# vis_regions = ['exon yc', 'intron yc', 'te yc', 'promoter yc', 'intergenic yc']
sub_norm_elem_rates = {'ind':dict(), 'gen':dict(), 'low':dict(), 'high':dict()}
sample_to_genotype = []
gen_colors = []

for sample in vis_samples:
    sample_genotype = sample.rsplit('-', 1)[0]
    sub_norm_elem_rates['ind'][sample] = {region:norm_elem_rates['ind'][sample][region] for region in vis_regions}
    
for genotype in vis_genotypes:
    sub_norm_elem_rates['gen'][genotype] = {region:norm_elem_rates['gen'][genotype][region] for region in vis_regions}
    sub_norm_elem_rates['low'][genotype] = {region:norm_elem_rates['low'][genotype][region] for region in vis_regions}
    sub_norm_elem_rates['high'][genotype] = {region:norm_elem_rates['high'][genotype][region] for region in vis_regions}


In [ ]:
def log2_min(x):
    if x == 0:
        return -2
    else:
        return math.log2(x)

In [ ]:
sems = {g:dict() for g in vis_genotypes}
for g in vis_genotypes:
    for region in vis_regions:
        # I tried bootstrapping sems here. It gives smaller values so I decided not to use it. The normal way of calculating sem is
        # probably overly conservative though, as the samples have quantized rates which make it impossible to have values right on the mean
        if g == 'MA lines': 
            boot_rates = []
            for b in range(2000):
                boot_samples = random.choices(g_to_s('MA lines'), k=len(g_to_s('MA lines')))
                region_count = sum([elem_counts['ind'][s][region] for s in boot_samples])
                total_count = sum([len(dfs_somatic[s]) for s in boot_samples])
                region_cov = sum([elem_cov['ind'][s][region] for s in boot_samples])
                total_cov = elem_cov['uniform'][g] # the total cov remains the same regardless of which samples were bootstrapped
                
                relative_rate = (region_count / region_cov) / (total_count / total_cov)
                boot_rates.append(relative_rate)
            boot_sem = np.std(boot_rates)
            normal_sem = np.std([norm_elem_rates["ind"][s][region] for s in g_to_s(g)]) / (len(g_to_s(g)) ** (1/2))
            
            print(f'{boot_sem:.4f} bootstrapped sem, {normal_sem:.4f} raw sem')
            sems[g][region] = normal_sem # just use the normal method instead of the bootstrap estimate
        else:
            # calculate using replicates
            sems[g][region] = np.std([norm_elem_rates['ind'][s][region] for s in g_to_s(g)]) / (len(g_to_s(g)) ** (1/2))

            # calculate using binomial (ignore biological differences in mutation distribution between replicates), FIXME I don't think this is right
            # count_sem = stats.binom.std(gen_count_sums[g], elem_counts['gen'][g][region] / gen_count_sums[g]) / (gen_count_sums[g] ** (1/2))
            # sems[g][region] = (count_sem / elem_cov['gen'][g][region]) / elem_rates['gen_uniform'][g]

In [ ]:
# plot log2 difference of mutation rate relative to uniform (e.g. a value of -1 means there are half as many mutations in the element as expected in a uniform distribution)
fig, ax = plt.subplots(figsize=(len(vis_regions) * (len(vis_genotypes) + 1) * 0.2, 3))
bars = []
for g in vis_genotypes:
    # low_err = [log2_min(sub_norm_elem_rates['gen'][g][t]) - log2_min(max(0.01, sub_norm_elem_rates['low'][g][t])) for t in vis_regions]
    # high_err = [log2_min(sub_norm_elem_rates['high'][g][t]) - log2_min(sub_norm_elem_rates['gen'][g][t]) for t in vis_regions]
    x = [x * (len(vis_genotypes) + 1) + vis_genotypes.index(g) for x in range(len(vis_regions))]
    y = [y - 1 for y in sub_norm_elem_rates['gen'][g].values()]
    bars.append(ax.bar(x, y, color=gen_cmap(g), alpha=0.6, bottom=1, yerr=sems[g].values()))
for s in vis_samples:
    g = s_to_g(s)
    x = [x * (len(vis_genotypes) + 1) + vis_genotypes.index(g) + random.random() * 0.5 - 0.25 for x in range(len(vis_regions))]
    y = sub_norm_elem_rates['ind'][s].values()
    ax.scatter(x, y, color=gen_cmap(g), s=4)
# ax.set_ylim(0.37, 2.9)
ax.set_yscale('log', base=2)
xlim = ax.get_xlim()
ax.hlines(1, -10, 100, color='black')
ax.set_xlim(xlim)
ax.set_xlim(ax.get_xlim()[0] - 0.5, ax.get_xlim()[1] + 0.5)

ax.set_xticks([i * (len(vis_genotypes) + 1) + (len(vis_genotypes) - 1) / 2 for i in range(len(vis_regions))])
ax.set_xticklabels(vis_regions, rotation=45)
p_labels = [f'p={p:.2f}' if p > 0.001 else f'p<e{math.floor(math.log10(max(10**-20, p)))}' for p in p_vals]
ax.legend(bars, vis_genotypes, ncol=1, loc='upper left', bbox_to_anchor=(0, 1))
ax.set_ylabel('Log2fold mutation rate')
# ax.set_title('Mutation rate by genomic region')
fig.savefig('figs/mutation_rate_by_element_type_log_fold.png', dpi=300, bbox_inches='tight')
fig.savefig('figs/mutation_rate_by_element_type_log_fold.svg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
for i in range(0, len(vis_regions) - 1, 2):
    for g in vis_genotypes:
        rate1 = norm_elem_rates['gen'][g][vis_regions[i]]
        rate2 = norm_elem_rates['gen'][g][vis_regions[i + 1]]
        print(f'{g} {vis_regions[i + 1]}/{vis_regions[i]} {rate2 / rate1:.4f}')

In [ ]:
pvals = []
paired_regions = True # whether vis_regions represent one or more pairs of regions to compare (e.g. these regions would be paired: mCG, umCG, mCHG, umCHG)


# for each region, compare col-0 to each other group with >1 sample
if not paired_regions: # compare col-0 bar to other genotype bar for each region and genotype FIXME put not back
    for region in vis_regions:
        for g in vis_genotypes:
            if g == 'untreated':
                continue
            if len(g_to_s(g)) > 1:
                # welch's two sample t-test
                p = stats.ttest_ind([norm_elem_rates['ind'][s][region] for s in g_to_s('untreated')], [norm_elem_rates['ind'][s][region] for s in g_to_s(g)], equal_var=False).pvalue

                ratio = norm_elem_rates['gen']['untreated'][region] / norm_elem_rates['gen'][g][region]
                pvals.append((p, f'{region} untreated  to {g} two-sample welchs ttest ({ratio:.4f}x or {1/ratio:.4f}x)'))
else: # compare ratio of bar1:bar2 for col-0 and other genotypes, then do the same for bar3:bar4...
    for i in range(0, len(vis_regions) - 1, 2):
        for g in vis_genotypes:
            if g == 'untreated' or len(g_to_s(g)) == 1:
                continue
            r1 = vis_regions[i]
            r2 = vis_regions[i + 1]
            p = stats.ttest_1samp([norm_elem_rates['ind'][s][r2] / norm_elem_rates['ind'][s][r1] for s in g_to_s('untreated')], norm_elem_rates['gen'][g][r2] / norm_elem_rates['gen'][g][r1]).pvalue
            ratios = (norm_elem_rates['gen']['untreated'][r2] / norm_elem_rates['gen']['untreated'][r1],  norm_elem_rates['gen'][g][r2] / norm_elem_rates['gen'][g][r1])
            pvals.append((p, f'{r1}/{r2} ratio differs between untreated to {g} one-sample ttest ({ratios[0]:.4f}x to {ratios[1]:.4f}x)'))
            
# compare pairs of regions for col-0 and ma lines

for i, r1 in enumerate(vis_regions[:-1]):
    if paired_regions and i % 2 == 1:
        continue
    comp_regions = [vis_regions[i + 1]] if paired_regions else vis_regions[i + 1:]
    for r2 in comp_regions:
        p = stats.ttest_ind([norm_elem_rates['ind'][s][r1] for s in g_to_s('untreated')], [norm_elem_rates['ind'][s][r2] for s in g_to_s('untreated')], equal_var=True).pvalue
        ratio = norm_elem_rates['gen']['untreated'][r1] / norm_elem_rates['gen']['untreated'][r2]
        pvals.append((p, f'untreated {r1} to {r2} two-sample ttest ({ratio:.4f}x or {1/ratio:.4f}x)'))
        
        p = stats.ttest_ind([norm_elem_rates['ind'][s][r1] for s in g_to_s('MA lines')], [norm_elem_rates['ind'][s][r2] for s in g_to_s('MA lines')], equal_var=True).pvalue
        ratio = norm_elem_rates['gen']['MA lines'][r1] / norm_elem_rates['gen']['MA lines'][r2]
        pvals.append((p, f'MA lines {r1} to {r2} two-sample ttest ({ratio:.4f}x or {1/ratio:.4f}x)'))

# # compare col-0 genes to MA line genes
# p = stats.ttest_ind([norm_elem_rates['ind'][s]['gene'] for s in g_to_s('untreated')], [norm_elem_rates['ind'][s]['gene'] for s in g_to_s('MA lines')], equal_var=False).pvalue
# pvals.append((p, f'genes Col-0 to MA lines two-sample Welchs ttest'))


In [ ]:
cutoff_reached = False
for i, pval in enumerate(sorted(pvals)):
    corrected_p = pval[0] * (len(pvals) - i)
    if corrected_p > 0.05 and not cutoff_reached:
        print('--------------- significance cutoff -----------------')
        cutoff_reached = True
    print(corrected_p, pval[1])

In [ ]:
# same as the above plot, but use the raw mutation rate rather than the relative rate
fig, ax = plt.subplots(figsize=(len(vis_regions) * (len(vis_genotypes) + 1) * 0.5, 3))
bars = []
for g in vis_genotypes:
    # low_err = [log2_min(sub_norm_elem_rates['gen'][g][t]) - log2_min(max(0.01, sub_norm_elem_rates['low'][g][t])) for t in vis_regions]
    # high_err = [log2_min(sub_norm_elem_rates['high'][g][t]) - log2_min(sub_norm_elem_rates['gen'][g][t]) for t in vis_regions]
    x = [x * (len(vis_genotypes) + 1) + vis_genotypes.index(g) for x in range(len(vis_regions))]
    y = [sub_norm_elem_rates['gen'][g][r] * elem_rates['gen_uniform'][g] for r in vis_regions]
    errors = [sems[g][r] * elem_rates['gen_uniform'][g] for r in vis_regions]
    bars.append(ax.bar(x, y, color=gen_cmap(g), alpha=0.6, bottom=0, yerr=errors))
for s in vis_samples:
    g = s_to_g(s)
    x = [x * (len(vis_genotypes) + 1) + vis_genotypes.index(g) + random.random() * 0.5 - 0.25 for x in range(len(vis_regions))]
    y = [sub_norm_elem_rates['ind'][s][r] * elem_rates['gen_uniform'][g] for r in vis_regions]
    ax.scatter(x, y, color=gen_cmap(g), s=4)
# ax.set_yscale('log', base=2)
xlim = ax.get_xlim()
# ax.hlines(1, -10, 100, color='black')
ax.set_xlim(xlim)
ax.set_xlim(ax.get_xlim()[0] - 0.5, ax.get_xlim()[1] + 0.5)
# ax.set_ylim(-1.5, 1.5)

ax.set_xticks([i * (len(vis_genotypes) + 1) + (len(vis_genotypes) - 1) / 2 for i in range(len(vis_regions))])
ax.set_xticklabels(vis_regions, rotation=45)
p_labels = [f'p={p:.2f}' if p > 0.001 else f'p<e{math.floor(math.log10(max(10**-20, p)))}' for p in p_vals]
ax.legend(bars, vis_genotypes, ncol=1, loc='upper left', bbox_to_anchor=(0, 1))
ax.set_ylabel('mutation rate')
# ax.set_title('Mutation rate by genomic region')
fig.savefig('figs/mutation_rate_by_element_type_raw.svg', dpi=300, bbox_inches='tight')
plt.show()

# Mutation rate over chromosomes

In [ ]:
yc_mask = {chrom:np.zeros(chrom_sizes[chrom], dtype=bool) for chrom in chrom_sizes}
for chrom in chrom_sizes:
    for i in tqdm(range(1, len(genome[chrom]) - 1)):
        first = genome[chrom][i]
        second = genome[chrom][i + 1]
        if first == 'G' and (second == 'G' or second == 'A'):
            yc_mask[chrom][i] = 1
        elif second == 'C' and (first == 'C' or first == 'T'):
            yc_mask[chrom][i + 1] = 1
    

In [ ]:
window_size = 2000000 # use these for the line plots
step_size = 200000

# window_size = 1000000 # use these for the scatter plots
# step_size = 1000000

# count up the amount of mutations and callable coverage in each bin of the genome for each genotype
chrom_muts = {g:{chrom:np.zeros(chrom_sizes[chrom] // step_size + 1, dtype=int) for chrom in chrom_sizes} for g in genotypes}
# chrom_muts_yc = {g:{chrom:np.zeros(chrom_sizes[chrom] // step_size + 1, dtype=int) for chrom in chrom_sizes} for g in genotypes}
chrom_covs = {g:{chrom:np.zeros(chrom_sizes[chrom] //  step_size + 1, dtype=int) for chrom in chrom_sizes} for g in genotypes}
# chrom_covs_yc = {g:{chrom:np.zeros(chrom_sizes[chrom] //  step_size + 1, dtype=int) for chrom in chrom_sizes} for g in genotypes}
total_chrom_cov = {g:{chrom:0 for chrom in chrom_sizes} for g in genotypes}
for chrom in chrom_sizes:
    ma_cov_arr = np.load(f'{cov_prefixes["untreated_1"]}{chrom}.npy') > 0
    # ma_cov_arr_yc = np.multiply(yc_mask[chrom], ma_cov_arr)
    
    for sample in tqdm(sample_names):
        # count mutations
        df = dfs_somatic[sample]
        genotype = s_to_g(sample)
        mut_arr = np.zeros(chrom_sizes[chrom], dtype=int)
        for pos in df[df.chrom == chrom].pos:
            mut_arr[pos] += 1
        # mut_arr_yc = np.multiply(yc_mask[chrom], mut_arr)
            
        # count callable coverage
        if 'Ler' in genotype: # use sites with >0 coverage
            cov_arr = np.load(f'{cov_prefixes[sample]}{chrom}.npy')
            cov_arr = cov_arr > 0
            # cov_arr_yc = np.multiply(yc_mask[chrom], cov_arr)
        elif '1001' in genotype or 'MA lines' in genotype: # use sites in Col-0-1 with >0 coverage
            cov_arr = ma_cov_arr
            # cov_arr_yc = ma_cov_arr_yc
        else:
            cov_arr = np.load(f'{cov_prefixes[sample]}{chrom}.npy')
            # cov_arr_yc = np.multiply(yc_mask[chrom], cov_arr)
        
        for i, center in enumerate(range(0, chrom_sizes[chrom], step_size)):
            chrom_muts[genotype][chrom][i] += np.sum(mut_arr[max(0, center - window_size // 2):center + window_size // 2])
            # chrom_muts_yc[genotype][chrom][i] += np.sum(mut_arr_yc[max(0, center - window_size // 2):center + window_size // 2])
            chrom_covs[genotype][chrom][i] += np.sum(cov_arr[max(0, center - window_size // 2):center + window_size // 2])
            # chrom_covs_yc[genotype][chrom][i] += np.sum(cov_arr_yc[max(0, center - window_size // 2):center + window_size // 2])
        total_chrom_cov[genotype][chrom] += np.sum(cov_arr)

chrom_boot_lower = {chrom:np.zeros(chrom_sizes[chrom] // step_size + 1, dtype=float) for chrom in chrom_sizes}
chrom_boot_upper = {chrom:np.zeros(chrom_sizes[chrom] // step_size + 1, dtype=float) for chrom in chrom_sizes}
for chrom in chrom_sizes:
    # get the number of mutations and callable coverage per bin for each of the 8 WT replicates
    mut_arrs = [np.zeros(chrom_sizes[chrom] // step_size + 1, dtype=int) for x in range(8)]
    cov_arrs = [np.zeros(chrom_sizes[chrom] // step_size + 1, dtype=int) for x in range(8)]
    for i in range(8):
        mut_arr = np.zeros(chrom_sizes[chrom], dtype=int)
        cov_arr = np.zeros(chrom_sizes[chrom], dtype=int)
        df = dfs_somatic[f'untreated_{i+1}']
        for pos in df[df.chrom == chrom].pos:
            mut_arr[pos] += 1
        cov_arr = np.load(f'{cov_prefixes["untreated_" + str(i+1)]}{chrom}.npy')
        
        for bin_idx, center in enumerate(range(0, chrom_sizes[chrom], step_size)):
            mut_arrs[i][bin_idx] = np.sum(mut_arr[max(0, center - window_size // 2):center + window_size // 2])
            cov_arrs[i][bin_idx] = np.sum(cov_arr[max(0, center - window_size // 2):center + window_size // 2])
            
    # for 2000 bootstraps of 8 samples, get a mutation rate
    boot_rates = np.zeros((2000, len(mut_arrs[0])), dtype=float)
    for b in range(2000):
        boot_samples = random.choices(range(8), k=8)
        mut_arr = np.sum([mut_arrs[x] for x in boot_samples], axis=0)
        cov_arr = np.sum([cov_arrs[x] for x in boot_samples], axis=0)
        boot_rates[b] = np.divide(mut_arr, cov_arr)
    
    lower_boots = np.percentile(boot_rates, 2.5, axis=0)
    upper_boots = np.percentile(boot_rates, 97.5, axis=0)
    assert len(lower_boots) == len(mut_arrs[0])
    chrom_boot_lower[chrom] = lower_boots
    chrom_boot_upper[chrom] = upper_boots
    
        
# get an upper and lower bound for mutation number assuming binomial distribution
chrom_muts_lower = {g:{chrom:np.zeros(chrom_sizes[chrom] // step_size + 1, dtype=int) for chrom in chrom_sizes} for g in genotypes}
chrom_muts_upper = {g:{chrom:np.zeros(chrom_sizes[chrom] // step_size + 1, dtype=int) for chrom in chrom_sizes} for g in genotypes}
for g in genotypes:
    total_muts = sum([sum(chrom_muts[g][chrom]) for chrom in chrom_sizes])
    for chrom in chrom_sizes:
        for i in range(len(chrom_muts[g][chrom])):
            lower, upper = stats.binom.interval(0.95, total_muts, max(1, chrom_muts[g][chrom][i]) / total_muts)
            chrom_muts_lower[g][chrom][i] = lower
            chrom_muts_upper[g][chrom][i] = upper
            
chrom_rates = dict()
chrom_rates_yc = dict()
chrom_rates_lower = dict()
chrom_rates_upper = dict()
chrom_rates_boot_lower = dict()
chrom_rates_boot_upper = dict()
for g in genotypes:
    total_muts = sum([sum(chrom_muts[g][chrom]) for chrom in chrom_sizes])
    # total_muts_yc = sum([sum(chrom_muts[g][chrom]) for chrom in chrom_sizes])
    total_cov = sum([sum(chrom_covs[g][chrom]) for chrom in chrom_sizes])
    # total_cov_yc = sum([sum(chrom_covs_yc[g][chrom]) for chrom in chrom_sizes])
    chrom_rates[g] = {chrom:(chrom_muts[g][chrom] / (chrom_covs[g][chrom] + 1)) / (total_muts / total_cov) for chrom in chrom_sizes}
    # chrom_rates_yc[g] = {chrom:(chrom_muts_yc[g][chrom] / (chrom_covs_yc[g][chrom] + 1)) / (total_muts_yc / total_cov_yc) for chrom in chrom_sizes}
    chrom_rates_lower[g] = {chrom:(chrom_muts_lower[g][chrom] / (chrom_covs[g][chrom] + 1)) / (total_muts / total_cov) for chrom in chrom_sizes}
    chrom_rates_upper[g] = {chrom:(chrom_muts_upper[g][chrom] / (chrom_covs[g][chrom] + 1)) / (total_muts / total_cov) for chrom in chrom_sizes}
    
    if g == 'untreated':
        chrom_rates_boot_lower = {chrom:chrom_boot_lower[chrom] / (total_muts / total_cov) for chrom in chrom_sizes}
        chrom_rates_boot_upper = {chrom:chrom_boot_upper[chrom] / (total_muts / total_cov) for chrom in chrom_sizes}
        print(total_muts,  total_cov)

In [ ]:
vis_genotypes = genotypes
vis_samples = [s for s in sample_names if s_to_g(s) in vis_genotypes]

import matplotlib.lines as mlines

# centromere_pos = {'Chr1':16029005, 'Chr2':5731747, 'Chr3':14711236, 'Chr4':6595103, 'Chr5':13790251} # don't remember what paper this is from, but it's supposed to be the center of the centromere repeats
centromere_pos = {'Chr1':15256434, 'Chr2':3697368, 'Chr3':13599663, 'Chr4':3941890, 'Chr5':11880011} # this is from the Col-CEN paper (34762468) table S1 and figure S3, center of the centromere
fig, axs = plt.subplots(len(chrom_sizes), figsize=(10, 6), sharex=True)
for i, chrom in enumerate(chrom_sizes):
    lines = []
    for genotype in vis_genotypes:
        l = axs[i].plot([x * step_size for x in range(len(chrom_rates[genotype][chrom]))], chrom_rates[genotype][chrom], c=gen_cmap(genotype))
        if genotype == 'untreated':
            # axs[i].fill_between([x * step_size for x in range(len(chrom_rates[genotype][chrom]))], chrom_rates_lower[genotype][chrom], chrom_rates_upper[genotype][chrom], color=gen_cmap(genotype), alpha=0.2, edgecolors='none')
            axs[i].fill_between([x * step_size for x in range(len(chrom_rates[genotype][chrom]))], chrom_rates_boot_lower[chrom], chrom_rates_boot_upper[chrom], color=gen_cmap(genotype), alpha=0.2, edgecolors='none')
        lines.append(l[0])
    ylim = axs[i].get_ylim()
    axs[i].vlines(centromere_pos[chrom], -1, axs[i].get_ylim()[1] + 1, color='black')
    axs[i].set_ylim(ylim)
    axs[i].set_title(chrom, y=0.6, x=0.95)
axs[-1].set_xlabel('position (bp)')
axs[0].set_xlim(0, 30000000)
axs[2].set_ylabel('Mutation/polymorphism rate relative to mean')
axs[0].legend(lines, vis_genotypes, loc='upper left', bbox_to_anchor=(1, 1))
# axs[0].legend(['untreated', '95% confidence (mutuations per bin ~ binomial)', '95% confidence (2000 bootstraps of 8 samples)'], bbox_to_anchor=(1,1))
fig.suptitle('Mutation rate across chromosomes')
fig.savefig('figs/mutation_rate_across_chromosomes.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.lines as mlines

# centromere_pos = {'Chr1':16029005, 'Chr2':5731747, 'Chr3':14711236, 'Chr4':6595103, 'Chr5':13790251} # don't remember what paper this is from, but it's supposed to be the center of the centromere repeats
centromere_pos = {'Chr1':15256434, 'Chr2':3697368, 'Chr3':13599663, 'Chr4':3941890, 'Chr5':11880011} # this is from the Col-CEN paper (34762468) table S1 and figure S3, center of the centromere
fig, axs = plt.subplots(len(chrom_sizes), figsize=(10, 6), sharex=True)
for i, chrom in enumerate(chrom_sizes):
    lines = []
    for genotype in vis_genotypes:
        l = axs[i].plot([x * step_size for x in range(len(chrom_rates_yc[genotype][chrom]))], chrom_rates_yc[genotype][chrom], c=gen_cmap(genotype))
        lines.append(l[0])
    ylim = axs[i].get_ylim()
    axs[i].vlines(centromere_pos[chrom], -1, axs[i].get_ylim()[1] + 1, color='black')
    axs[i].set_ylim(ylim)
    axs[i].set_title(chrom, y=0.6, x=0.95)
axs[-1].set_xlabel('position (bp)')
axs[0].set_xlim(0, 30000000)
axs[2].set_ylabel('Relative mutation rate of YC sites')
axs[0].legend(lines, vis_genotypes, loc='upper left', bbox_to_anchor=(1, 1))
# axs[0].legend(['untreated', '95% confidence (mutuations per bin ~ binomial)', '95% confidence (2000 bootstraps of 8 samples)'], bbox_to_anchor=(1,1))
fig.suptitle('Mutation rate of YC sites across chromosomes')
fig.savefig('figs/mutation_rate_across_chromosomes_yc.svg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# centromere_pos = {'Chr1':16029005, 'Chr2':5731747, 'Chr3':14711236, 'Chr4':6595103, 'Chr5':13790251} # don't remember what paper this is from, but it's supposed to be the center of the centromere repeats
centromere_pos = {'Chr1':15256434, 'Chr2':3697368, 'Chr3':13599663, 'Chr4':3941890, 'Chr5':11880011} # this is from the Col-CEN paper (34762468) table S1 and figure S3, center of the centromere
fig, axs = plt.subplots(len(chrom_sizes), figsize=(10, 6), sharex=True, sharey=True)
for i, chrom in enumerate(chrom_sizes):
    lines = []
    for genotype in vis_genotypes:
        x = [x * step_size for x in range(len(chrom_covs[genotype][chrom]))]
        x_sizes = np.array([min(chrom_sizes[chrom], center + window_size // 2) - max(0, center - window_size // 2) for center in x])
        # print(chrom_covs[genotype][chrom] / x_sizes)
        # print(total_chrom_cov[genotype] / chrom_sizes[chrom])
        # print(chrom_covs[genotype][chrom])
        y = (chrom_covs[genotype][chrom] / x_sizes) / (total_chrom_cov[genotype][chrom] / chrom_sizes[chrom]) # coverage per site / genome average
        l = axs[i].plot(x, y, c=gen_cmap(genotype))
        lines.append(l[0])
    ylim = axs[i].get_ylim()
    axs[i].vlines(centromere_pos[chrom], -1, axs[i].get_ylim()[1] + 1, color='black')
    axs[i].set_ylim(ylim)
    axs[i].set_title(chrom, y=0.6, x=0.95)
axs[-1].set_xlabel('position (bp)')
axs[0].set_xlim(0, 30000000)
axs[0].set_ylim(0, axs[0].get_ylim()[1])
axs[2].set_ylabel('Callable coverage relative to mean')
axs[0].legend(lines, vis_genotypes, loc='upper left', bbox_to_anchor=(1, 1))
# axs[0].legend(['untreated', '95% confidence (mutuations per bin ~ binomial)', '95% confidence (2000 bootstraps of 8 samples)'], bbox_to_anchor=(1,1))
fig.suptitle('Callable coverage across chromosomes')
fig.savefig('figs/callable_coverage_across_chromosomes.svg', dpi=300, bbox_inches='tight')
plt.show()

### Scatterplot best fit lines of the above data

In [ ]:
# get the distance of each bin from the centromere
bin_dists = {chrom:[abs(step_size * i - centromere_pos[chrom]) for i in range(chrom_sizes[chrom] // step_size + 1)] for chrom in chrom_sizes}

In [ ]:
# visualize correlation of distance from centromere with substitution rate
cols = int(len(vis_genotypes) ** (1/2))
fig, axs = plt.subplots(len(vis_genotypes) // cols, cols, figsize=(len(vis_genotypes), len(vis_genotypes)), sharex=True)

for i, g in enumerate(vis_genotypes):
    if cols > 1:
        ax = axs[i // cols, i % cols]
    else:
        ax = axs[i]
    
    x = []
    y = []
    for chrom in chrom_sizes:
        x += bin_dists[chrom]
        y += list(chrom_rates[g][chrom])
    
    slope, intersect = np.polyfit(x, y, 1)
    color = gen_cmap(g)
    r = np.corrcoef(x, y)
    ax.scatter(x, y, color=color, s=5)
    ax.axline((0, intersect), slope=slope, color=color)
    ax.set_title(f'{g} r={r[0][1]:.2f}', y=0.8)
    ax.set_ylim(0, ax.get_ylim()[1])
    ax.set_ylabel('Relative substitution rate')
    ax.set_xlabel('Distance from centromere')
    
fig.suptitle('Substitution rate correlates with distance from centromere')
fig.savefig('figs/dist_from_centromere_scatters.svg', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# YC sites only
cols = int(len(vis_genotypes) ** (1/2))
fig, axs = plt.subplots(len(vis_genotypes) // cols, cols, figsize=(len(vis_genotypes), len(vis_genotypes)), sharex=True)

for i, g in enumerate(vis_genotypes):
    ax = axs[i // cols, i % cols]
    
    x = []
    y = []
    for chrom in chrom_sizes: #chrom_sizes: # FIXME
        x += bin_dists[chrom]
        y += list(chrom_rates_yc[g][chrom])
    
    slope, intersect = np.polyfit(x, y, 1)
    color = gen_cmap(g)
    r = np.corrcoef(x, y)
    ax.scatter(x, y, color=color, s=5)
    ax.axline((0, intersect), slope=slope, color=color)
    ax.set_title(f'{g} r={r[0][1]:.2f}', y=0.8)
    ax.set_ylim(0, ax.get_ylim()[1])
    
axs[len(axs) // 2][0].set_ylabel('Relative mutation rate of YC sites')
axs[-1][cols // 2].set_xlabel('Distance from centromere')
fig.suptitle('mutation rate and distance to centromere (YC sites only)')
fig.savefig('figs/dist_from_centromere_scatters_yc.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
len(bin_dists['Chr2']), len(chrom_rates[genotypes[0]]['Chr2'])

# Mutation hotspots

In [ ]:
hotspot_size = 10000

# make a histogram of the number of mutations within half-overlapping sliding windows of the genome
obs_hist = {g:np.zeros(10000) for g in genotypes}
genotype_muts = {g:0 for g in genotypes} # total number of mutations for the genotype
for g in tqdm(genotypes):
    # make arrays of the mutations per site for the genotype
    obs_muts = {chrom:np.zeros(chrom_sizes[chrom], dtype=np.ubyte) for chrom in chrom_sizes}
    samples = [s for s in sample_names if s_to_g(s) == g] # samples belonging to genotype
    for s in samples:
        for row in dfs_somatic[s].itertuples():
            obs_muts[row.chrom][row.pos] += 1
            genotype_muts[g] += 1
    
    # iterate over sliding windows, find the mutation count in the window and increment obs_hist
    for chrom in chrom_sizes:
        i = 0
        while i < chrom_sizes[chrom]:
            muts = np.sum(obs_muts[chrom][i:i + hotspot_size])
            obs_hist[g][muts] += 1
            i += hotspot_size // 2

In [ ]:
# calculated the expected histogram as a binomial distribution
exp_hist = {g:np.zeros(10000) for g in genotypes}
for g in genotypes:
    for i in range(len(exp_hist[g])):
        exp_hist[g][i] = stats.binom.pmf(i, genotype_muts[g], hotspot_size / sum(chrom_sizes.values())) * np.sum(obs_hist[g])

In [ ]:
vis_genotypes = ['untreated', 'UVB', 'UVC', 'MA lines']
fig, axs = plt.subplots(len(vis_genotypes), figsize=(8, len(vis_genotypes) * 1.5), gridspec_kw={'hspace':0.3}, sharex=True)
for i, g in enumerate(vis_genotypes):
    max_x = np.max(np.nonzero(obs_hist[g])) + 10
    axs[i].hlines(1, -1, max_x + 2, color='black')
    axs[i].plot(range(max_x), obs_hist[g][:max_x], '-o', color=gen_cmap(g))
    axs[i].plot(range(max_x), exp_hist[g][:max_x], '-o', color='tab:grey')
    axs[i].set_title(g, y=0.6)
    axs[i].set_yscale('log')
    # print(axs[i].get_ylim(), np.max(obs_hist[g][:max_x]), np.max(exp_hist[g][:max_x]))
    axs[i].set_ylim(0.01, np.max(exp_hist[g]) * 1.5)
    axs[i].set_xlim(-0.3, max_x - 0.7)
axs[-1].set_xlabel('number of mutations in window')
axs[len(axs) // 2].set_ylabel('count')
fig.savefig('figs/hotspots_for_mutations.svg', dpi=300, bbox_inches='tight')

# Logistic regression

this section is very memory heavy, requiring ~400GB when doing regression on the whole genome

In [ ]:
del df_gff
for r in ['intron', 'exon', 'te', 'promoter', 'intergenic', 'unmet cg', 'met cg', 'unmet chg', 'met chg', 'unmet chh', 'met chh']:
    del region_arrays[r]

In [ ]:
import statsmodels.api as sm
import time

In [ ]:
# used for converting chrom:pos positions to just a single index of a 1D array
chrom_offset = dict()
tot = 0
for chrom in chrom_sizes:
    chrom_offset[chrom] = tot
    tot += chrom_sizes[chrom]

In [ ]:
reg_genotype = '1001'
reg_region = None

In [ ]:
cg = np.concatenate(list(region_arrays['cg'].values())) # note that this is not CG vs AT, but CG vs AT/NN/YR/SS/WW
peri = np.concatenate(list(region_arrays['pericentromere'].values()))
met = np.concatenate(list(region_arrays['met c'].values())) # note that this is not met vs unmet, but met vs unmet/unknown
acr = np.concatenate(list(region_arrays['acr'].values()))
genic = np.concatenate(list(region_arrays['gene'].values()))
ns = np.concatenate(list(ns_sites.values())) # note that this is not nonsyn vs syn, but nonsyn vs syn/nonCDS

intercept = np.ones(len(cg), dtype=bool) # statsmodels does not use an intercept by default, so a column of 1s needs to be added to the explanatory variables

# make an array of predictor variables (reg_x)
if 'Ler-0' in reg_genotype or '1001' in reg_genotype:
    reg_x = np.vstack((intercept, cg, peri, met, acr, genic, acr * genic, ns)).T # explanatory variables, each column is one variable (e.g. methylation)
    predictors = ['intercept', 'cg', 'peri', 'met', 'acr', 'genic', 'acr*genic', 'ns']
else:
    reg_x = np.vstack((intercept, cg, peri, met, acr, genic, acr * genic)).T # explanatory variables, each column is one variable (e.g. methylation)
    predictors = ['intercept', 'cg', 'peri', 'met', 'acr', 'genic', 'acr*genic']

del cg, peri, met, acr, ns, intercept, genic


# count the number of mutations at each site in the genome (i.e. the successes of a binomial)
reg_successes = np.zeros(len(reg_x), dtype=int)
for s in sample_names:
    if s_to_g(s) != reg_genotype:
        continue
    for r in dfs_somatic[s].itertuples():
        reg_successes[chrom_offset[r.chrom] + r.pos] += 1

# count the amount of coverage at each site in the genome (i.e. the trials of a binomial)
if 'Ler' in reg_genotype: # get all sites with at least one coverage in ler
    arr = np.zeros(0)
    for chrom in chrom_sizes:
        arr = np.append(arr, np.load(f'{cov_prefixes["Ler-0"]}{chrom}.npy'))
    arr = arr > 0
elif '1001' in reg_genotype or 'MA lines' in reg_genotype: # get all sites with at least one coverage in Col-0-1
    arr = np.zeros(0)
    for chrom in chrom_sizes:
        arr = np.append(arr, np.load(f'{cov_prefixes["untreated_1"]}{chrom}.npy'))
    arr = arr > 0
else: # get the sum of callable coverage across samples
    arr = np.zeros(sum(chrom_sizes.values()))
    for s in sample_names:
        if s_to_g(s) != reg_genotype:
            continue
        for chrom in chrom_sizes:
            arr[chrom_offset[chrom]:chrom_offset[chrom] + chrom_sizes[chrom]] += np.load(f'{cov_prefixes[s]}{chrom}.npy')
reg_trials = arr

# restrict x and y to one region (e.g. run the model on promoters only)
if reg_region is not None:
    restrict = np.concatenate(list(region_arrays[reg_region].values()))
    reg_x = reg_x[restrict]
    reg_successes = reg_successes[restrict]
    reg_trials = reg_trials[restrict]
print(len(reg_x))

In [ ]:
# only use sites where there was callable coverage (the model cannot handle nan values)
x_subset = reg_x[reg_trials > 0]
success_subset = reg_successes[reg_trials > 0]
trial_subset = reg_trials[reg_trials > 0]

In [ ]:
if 'MA lines' in reg_genotype or 'Ler-0' in reg_genotype:
    y_subset = np.clip(success_subset, None, 1).T # only allow one mutation per site
    model = sm.Logit(y_subset, x_subset) # use logistic regression, as each site has a chance of containing a mutation or not
elif '1001' in reg_genotype:
    y_subset = success_subset.T
    model = sm.Poisson(y_subset, x_subset) # use poisson regression, as each site can have different mutations in different accessions (not a great model, as we can't distinguish double hits)
else:
    y_subset = np.vstack((success_subset, trial_subset - success_subset)).T
    model = sm.GLM(y_subset, x_subset, family=sm.families.Binomial()) # usually takes 15 mins on the whole genome
    
res = model.fit()
print(res.summary())

In [ ]:
# get the effect size of each explanatory variable
means = np.mean(x_subset, axis=0) # get mean of each column (fraction of genome with each flag)
for i, var in enumerate(predictors):
    arr = np.vstack((means, means)) # it's not necessary to use the means when gettingthe ratio of p[0]:p[1]
    arr[0][i] = 0
    arr[1][i] = 1
    p = res.predict(arr)
    print(f'predicted effect of {var}: {p[0]:.3e} -> {p[1]:.3e} ({p[1] / p[0]:.3f}x)')

In [ ]:
res.save(f'tmp/regression_{reg_genotype}_{reg_region}_{"".join([x[0] for x in predictors])}.pkl')

In [ ]:
from statsmodels.iolib.smpickle import load_pickle
models = {'untreated':load_pickle('tmp/regression_untreated_None_icpmaga.pkl'),
          'MA lines':load_pickle('tmp/regression_MA lines_None_icpmaga.pkl'),
          'Ler-0':load_pickle('tmp/regression_Ler-0_None_icpmagan.pkl'),
          '1001':load_pickle('tmp/regression_1001_None_icpmagan.pkl')}

In [ ]:
np.e**-17.7484

In [ ]:
models['untreated'].conf_int()
models['untreated'].params

In [ ]:
# for model in models:
#     print(model)
#     print(models[model].summary())
#     row = ''
#     for i in range(len(models[model].params)):
#         coef = models[model].params[i]
#         interval = models[model].conf_int()[i]
#         if coef < -2:
#             row += f'{np.e ** coef:.2e} [{np.e ** interval[0]:.2e}, {np.e ** interval[1]:.2e}]\n'
#         else:
#             row += f'{np.e ** coef:.2f} [{np.e ** interval[0]:.2f}, {np.e ** interval[1]:.2f}]\n'
            
#     print(row)

In [ ]:
avg_rates = dict()

In [ ]:
# calculate the genome wide average rate for the model. used to calculate relative rate
# this code block needs to be run multiple times after updating reg_genotype and x_subset above
# I'm not super sure why these don't match gen_rates, they do for MA lines and 1001 but not Ler-0
pred_vals = models[reg_genotype].predict(x_subset)
avg_rate = np.dot(pred_vals, trial_subset) / np.sum(trial_subset)
avg_rates[reg_genotype] = avg_rate
print(reg_genotype, avg_rate)

In [ ]:
avg_rates

In [ ]:
acr_genic = dict()
acr_nongenic = dict()
nonacr_genic = dict()
nonacr_nongenic = dict()

means = np.mean(x_subset, axis=0)
for g in models:
    x = means
    if '1001' in g or 'Ler' in g:
        x = means.copy()
    else:
        x = means[:-1].copy()
    
    covar = models[g].cov_params()
    
    avg_rate = avg_rates[g]
    
    # genic ACR rate
    x[4] = 1
    x[5] = 1
    x[6] = 1
    pred = models[g].predict(x) # predict the mutation rate for genic ACRs
    gradient = (pred * (1-pred) * x.T).T # I don't really know how this math works, but it's using the delta method to estimate a confidence interval assuming normal dist
    std_err = np.sqrt(np.dot(np.dot(gradient, covar), gradient))
    # acr_genic[g] = [pred[0] / avg_rate, 1.96 * std_err / avg_rate]
    acr_genic[g] = [math.log2(pred[0] / avg_rate), math.log2(pred[0] / avg_rate) - math.log2(pred[0] / avg_rate - 1.0 * std_err / avg_rate)]
    
    # nongenic ACR rate
    x[4] = 1
    x[5] = 0
    x[6] = 0
    pred = models[g].predict(x)
    gradient = (pred * (1-pred) * x.T).T
    std_err = np.sqrt(np.dot(np.dot(gradient, covar), gradient))
    # acr_nongenic[g] = [pred[0] / avg_rate, 1.96 * std_err / avg_rate]
    acr_nongenic[g] = [math.log2(pred[0] / avg_rate), math.log2(pred[0] / avg_rate) - math.log2(pred[0] / avg_rate - 1.0 * std_err / avg_rate)]
    
    # genic nonACR rate
    x[4] = 0
    x[5] = 1
    x[6] = 0
    pred = models[g].predict(x)
    gradient = (pred * (1-pred) * x.T).T
    std_err = np.sqrt(np.dot(np.dot(gradient, covar), gradient))
    # nonacr_genic[g] = [pred[0] / avg_rate, 1.96 * std_err / avg_rate]
    nonacr_genic[g] = [math.log2(pred[0] / avg_rate), math.log2(pred[0] / avg_rate) - math.log2(pred[0] / avg_rate - 1.0 * std_err / avg_rate)]
    
    # nongenic nonACR rate
    x[4] = 0
    x[5] = 0
    x[6] = 0
    pred = models[g].predict(x)
    gradient = (pred * (1-pred) * x.T).T
    std_err = np.sqrt(np.dot(np.dot(gradient, covar), gradient))
    # nonacr_nongenic[g] = [pred[0] / avg_rate, 1.96 * std_err / avg_rate]
    nonacr_nongenic[g] = [math.log2(pred[0] / avg_rate), math.log2(pred[0] / avg_rate) - math.log2(pred[0] / avg_rate - 1.0 * std_err / avg_rate)]
    

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
bars = []
for i, g in enumerate(models):
    x = [0 + 0.1 * (i - 2), 1 + 0.1 * (i - 2)]
    y = [acr_genic[g][0], nonacr_genic[g][0]]
    bar = ax.errorbar(x, y, fmt='o-', yerr=[[acr_genic[g][1], nonacr_genic[g][1]], [acr_genic[g][1], nonacr_genic[g][1]]], color=gen_cmap(g))
    bars.append(bar)
    
    x = [2 + 0.1 * (i - 2), 3 + 0.1 * (i - 2)]
    y = [acr_nongenic[g][0], nonacr_nongenic[g][0]]
    ax.errorbar(x, y, fmt='o-', yerr=[[acr_nongenic[g][1], nonacr_nongenic[g][1]], [acr_nongenic[g][1], nonacr_nongenic[g][1]]], color=gen_cmap(g))

ax.hlines(0, -1, 5, color='black')
ax.set_xlim(-0.3, 3.3)
ax.set_xticks(range(4), labels=['ACR genic', 'nonACR genic', 'ACR nongenic', 'nonACR nongenic'])
ax.set_ylabel('log2fold relative substitution rate')
ax.legend(bars, models)
fig.savefig('figs/regression_acr_effect.svg', dpi=300, bbox_inches='tight')

In [ ]:
acr_genic, nonacr_genic

In [ ]:
def load_chrom_sizes(index):
    chrom_sizes = dict()
    with open(index, 'r') as f:
        for l in f:
            split = l[:-1].split('\t')
            chrom_sizes[split[0]] = int(split[1])
    return chrom_sizes

In [ ]:
chrom_sizes = load_chrom_sizes('data/ref/ref.fa.fai')
chrom_sizes

In [ ]:
cg_sites = {chrom:np.zeros(chrom_sizes[chrom], dtype=bool) for chrom in chrom_sizes}

In [ ]:
cg_sites

In [ ]:
with open('data/methyl/bewick_bisulfite_1.allc', 'r') as f:
    for l in f:
        l = l.split('\t')
        chrom = l[0]
        pos = l[1]
        cg_sites[chrom][int(pos)] = True

In [ ]:
cg_sites['Chr2'][15]

In [ ]:
cg_sites

In [ ]:
cgs_near = {chrom:np.zeros(chrom_sizes[chrom], dtype=int) for chrom in chrom_sizes}
for chrom in chrom_sizes:
    for i in tqdm(range(chrom_sizes[chrom])):
        cgs_near[chrom][i] = sum(cg_sites[chrom][i - 50:i + 50])

In [ ]:
cgs_near = {chrom:np.zeros(chrom_sizes[chrom], dtype=int) for chrom in chrom_sizes}
with open('data/methyl/bewick_bisulfite_1.allc', 'r') as f:
    for l in tqdm(f):
        l = l.split('\t')
        chrom = l[0]
        pos = l[1]
        cgs_near[chrom][int(pos) - 50:int(pos) + 50] += 1

In [ ]:
cgs_near

In [ ]:
cgs_near['Chr1'] > 10

In [ ]:
# # calculate the mutation rate at CG vs AT sites
# samples = [s for s in sample_names if s_to_g(s) == 'Col-0']
# cg_muts = 0
# at_muts = 0
# for s in samples:
#     cg_muts += sum(dfs_somatic[s].ref == 'C') + sum(dfs_somatic[s].ref == 'G')
#     at_muts += sum(dfs_somatic[s].ref == 'A') + sum(dfs_somatic[s].ref == 'T')

# cg_cov = 0
# at_cov = 0
# for chrom in tqdm(chrom_sizes):
#     cg_arr = np.zeros(chrom_sizes[chrom], dtype=bool)
#     at_arr = np.zeros(chrom_sizes[chrom], dtype=bool)
#     for i, x in enumerate(genome[chrom]):
#         if x == 'C' or x == 'G':
#             cg_arr[i] = True
#         elif x == 'A' or x == 'T':
#             at_arr[i] = True
#     cov_arr = np.load(f'data/coverage/2f1r/big_Col-0-1_{chrom}.npy')
#     cg_cov += np.sum(cg_arr * cov_arr)
#     at_cov += np.sum(at_arr * cov_arr)

In [ ]:
effect_size = {x:[dict(), dict()] for x in 'cg pericentromere met acr'.split()}
# total_effect_size = {x:0 for x in 'cg pericentromere methylated acr'.split()}
effect_regions = 'exon intron te promoter intergenic'.split()

for r in effect_regions:
    # CG content
    effect_size['cg'][True][r] = norm_elem_rates['gen']['Col-0'][f'{r} cg']
    effect_size['cg'][False][r] = norm_elem_rates['gen']['Col-0'][f'{r} at']
    # total_effect_size[r] += norm_elem_rates['gen']['Col-0'][f'{r} cg'] / norm_elem_rates['gen']['Col-0'][f'{r} at']
    
    # pericentromere
    effect_size['pericentromere'][True][r] = norm_elem_rates['gen']['Col-0'][f'{r} pericentromere']
    effect_size['pericentromere'][False][r] = norm_elem_rates['gen']['Col-0'][f'{r} distal']
    
    # methylated
    effect_size['met'][True][r] = norm_elem_rates['gen']['Col-0'][f'{r} met']
    effect_size['met'][False][r] = norm_elem_rates['gen']['Col-0'][f'{r} unmet']
    
    # acr
    effect_size['acr'][True][r] = norm_elem_rates['gen']['Col-0'][f'{r} acr']
    effect_size['acr'][False][r] = norm_elem_rates['gen']['Col-0'][f'{r} nonacr']

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True)
for i, effect in enumerate(effect_size):
    ax = axs[i // 2][i % 2]
    for j, r in enumerate(effect_regions):
        ax.plot((0, 1), (math.log2(effect_size[effect][False][r]), math.log2(effect_size[effect][True][r])), marker='o', color=plt.get_cmap('tab10')(j))
        ax.set_title(effect + '?', y=0.8)
ax.set_xticks([0, 1], labels=['no', 'yes'])
axs[0][1].legend(effect_regions, loc='upper left', bbox_to_anchor=(1, 1))
axs[0][0].set_ylabel('log2fold relative substitution rate')
fig.suptitle('Breakdown of effects by region')
fig.savefig('figs/breakdown_of_effects_by_region.svg', dpi=300, bbox_inches='tight')

In [ ]:
# elem_cov['gen']['Col-0']

In [ ]:
total_effect_size = dict()
weights = [elem_counts['gen']['Col-0'][r] for r in effect_regions]
for effect in effect_size:
    # get the log2fold difference in relative rate between bases with the effect for each region
    relative_rates = [math.log2(effect_size[effect][True][r] / effect_size[effect][False][r]) for r in effect_regions]
    
    # do a weighted average of the log2fold differences (I'm using log2fold because averaging pure rates would inflate the effect of increasing rates)
    avg = sum([relative_rates[i] * (weights[i] / sum(weights)) for i in range(len(relative_rates))])
    total_effect_size[effect] = 2 ** avg # undo the log2
total_effect_size['selection'] = dnds['Ler-0']['ratio']
total_effect_size

In [ ]:
# acr_relative_rate = norm_elem_rates['gen']['Col-0']['ACRs'] / norm_elem_rates['gen']['Col-0']['not ACRs']
# met_relative_rate = norm_elem_rates['gen']['Col-0']['non-genic mC'] / norm_elem_rates['gen']['Col-0']['non-genic unmet C']
# cg_relative_rate = (cg_muts / cg_cov) / (at_muts / at_cov)
# # cg_relative_rate = (((simple_spectra['Col-0']['CA'] + simple_spectra['Col-0']['CG'] + simple_spectra['Col-0']['CT']) / simple_context_freq['C']) /
# #                     ((simple_spectra['Col-0']['TA'] + simple_spectra['Col-0']['TC'] + simple_spectra['Col-0']['TG']) / simple_context_freq['T'])) # FIXME need to include indels here


In [ ]:
simulations = {effect:{chrom:np.ones(chrom_sizes[chrom], dtype=float) for chrom in chrom_sizes} for effect in list(total_effect_size.keys()) + ['no selection', 'all']}

for chrom in tqdm(chrom_sizes):
    # cg_arr = np.zeros(chrom_sizes[chrom], dtype=bool)
    # for i, x in enumerate(genome[chrom]):
    #     if x == 'C' or x == 'G':
    #         cg_arr[i] = True
    simulations['cg'][chrom] = np.ones(chrom_sizes[chrom], dtype=float) * (total_effect_size['cg'] * region_arrays['cg'][chrom] + ~region_arrays['cg'][chrom])
    simulations['pericentromere'][chrom] = np.ones(chrom_sizes[chrom], dtype=float) * (total_effect_size['pericentromere'] * region_arrays['pericentromere'][chrom] + ~region_arrays['pericentromere'][chrom])
    simulations['met'][chrom] = np.ones(chrom_sizes[chrom], dtype=float) * (total_effect_size['met'] * region_arrays['met c'][chrom] + ~region_arrays['met c'][chrom])
    simulations['acr'][chrom] = np.ones(chrom_sizes[chrom], dtype=float) * (total_effect_size['acr'] * region_arrays['acr'][chrom] + ~region_arrays['acr'][chrom])
    simulations['selection'][chrom] = np.ones(chrom_sizes[chrom], dtype=float) * (total_effect_size['selection'] * ns_sites[chrom] + ~ns_sites[chrom])
    
    simulations['no selection'][chrom] = simulations['cg'][chrom] * simulations['pericentromere'][chrom] * simulations['met'][chrom] * simulations['acr'][chrom]
    simulations['all'][chrom] = simulations['no selection'][chrom] * simulations['selection'][chrom]
    
    # simulations[1][chrom] = simulations[0][chrom] * (cg_relative_rate  * cg_arr +                               ~cg_arr)
    # simulations[2][chrom] = simulations[1][chrom] * (acr_relative_rate * region_arrays['ACRs'][chrom] +         ~region_arrays['ACRs'][chrom])
    # simulations[3][chrom] = simulations[2][chrom] * (met_relative_rate * region_arrays['genic mCG'][chrom] +    ~region_arrays['genic mCG'][chrom]) # FIXME this needs to be all methylated sites
    # simulations[3][chrom] = simulations[3][chrom] * (met_relative_rate * region_arrays['non-genic mC'][chrom] + ~region_arrays['non-genic mC'][chrom])
    # simulations[4][chrom] = simulations[3][chrom] * (ns_relative_rate  * ns_sites[chrom] +                      ~ns_sites[chrom])

In [ ]:
vis_regions = ['exon', 'intron', 'te', 'promoter', 'intergenic'] # FIXME do I need to care about callable coverage here?

# get the genome-wide average mutation rate for each simulation
sim_uniforms = dict()
for effect in simulations:
    sim_uniforms[effect] = sum([np.sum(simulations[effect][chrom]) for chrom in chrom_sizes]) / sum(chrom_sizes.values())

# get the mutation rate relative to uniform for each region in each simulation
sim_rates = {effect:[] for effect in simulations}
for effect in simulations:
    for region in vis_regions:
        rate_sum = sum([np.sum(simulations[effect][chrom] * region_arrays[region][chrom]) for chrom in chrom_sizes])
        region_size = sum([np.count_nonzero(region_arrays[region][chrom]) for chrom in chrom_sizes])
        sim_rates[effect].append(rate_sum / region_size / sim_uniforms[effect])

In [ ]:
# # same as above but pretend I sequenced the simulated samples and had to factor in callable coverage
# vis_regions = ['exon', 'intron', 'te', 'promoter', 'intergenic'] # FIXME do I need to care about callable coverage here?

# # get the genome-wide average mutation rate for each simulation
# sim_uniforms = []
# for sim in simulations:
#     sim_uniforms.append(sum([np.sum(sim[chrom]) for chrom in chrom_sizes]) / sum(chrom_sizes.values()))

# # get the mutation rate relative to uniform for each region in each simulation
# sim_rates = [[] for x in simulations]
# for i, sim in enumerate(simulations):
#     for region in vis_regions:
#         rate_sum = sum([np.sum(sim[chrom] * region_arrays[region][chrom]) for chrom in chrom_sizes])
#         region_size = sum([np.count_nonzero(region_arrays[region][chrom]) for chrom in chrom_sizes])
#         sim_rates[i].append(rate_sum / region_size / sim_uniforms[i])

In [ ]:
len(x), len(y), len(low_err), len(high_err)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

x = [(x + 1) * 5 + 0 for x in range(len(vis_regions))]
y = [log2_min(x) for x in sim_rates['no selection']]
# print(x)
ax.bar(x, y, color=gtools.base_cmap[4])
    
# for i, t in enumerate(vis_regions):
low_err = [log2_min(norm_elem_rates['gen']['Col-0'][r]) - log2_min(max(0.01, norm_elem_rates['low']['Col-0'][r])) for r in vis_regions]
high_err = [log2_min(norm_elem_rates['high']['Col-0'][r]) - log2_min(norm_elem_rates['gen']['Col-0'][r]) for r in vis_regions]
x = [(x + 1) * 5 + 1 for x in range(len(vis_regions))]
y = [log2_min(norm_elem_rates['gen']['Col-0'][r]) for r in vis_regions]
ax.bar(x, y, yerr=[low_err, high_err], color=gen_cmap('Col-0'))

x = [(x + 1) * 5 + 2 for x in range(len(vis_regions))]
y = [log2_min(x) for x in sim_rates['all']]
# print(x)
ax.bar(x, y, color=gtools.light_cmap[4])

low_err = [log2_min(norm_elem_rates['gen']['Ler-0'][r]) - log2_min(max(0.01, norm_elem_rates['low']['Ler-0'][r])) for r in vis_regions]
high_err = [log2_min(norm_elem_rates['high']['Ler-0'][r]) - log2_min(norm_elem_rates['gen']['Ler-0'][r]) for r in vis_regions]
x = [(x + 1) * 5 + 3 for x in range(len(vis_regions))]
y = [log2_min(norm_elem_rates['gen']['Ler-0'][r]) for r in vis_regions]
ax.bar(x, y, yerr=[low_err, high_err], color=gen_cmap('Ler-0'))
    
    
ax.set_xticks([(x + 1) * 5 + 3 / 2 for x in range(len(vis_regions))], vis_regions)
ax.legend(['simulation no selection', 'untreated', 'simulation with selection', 'Ler-0'])

xlim = ax.get_xlim()
ax.hlines(0, -10, 100, color='black')
ax.set_xlim(xlim)
ax.set_ylabel('log2fold relative substitution rate')
fig.savefig('figs/simulated_mutation_distributions.svg', dpi=300, bbox_inches='tight')

In [ ]:
# old version that plotted all simulations
# fig, ax = plt.subplots(figsize=(8, 4))
# for i, effect in enumerate(simulations):
#     color = plt.get_cmap('tab20b')(i + 15) if i > 0 else 'black'
#     x = [(x + 1) * (len(simulations) + 2) + i for x in range(len(vis_regions))]
#     y = [log2_min(x) for x in sim_rates[effect]]
#     # print(x)
#     ax.bar(x, y, color=color)
    
# # for i, t in enumerate(vis_regions):
# low_err = [log2_min(norm_elem_rates['gen']['Col-0'][r]) - log2_min(max(0.01, norm_elem_rates['low']['Col-0'][r])) for r in vis_regions]
# high_err = [log2_min(norm_elem_rates['high']['Col-0'][r]) - log2_min(norm_elem_rates['gen']['Col-0'][r]) for r in vis_regions]
# x = [(x + 1) * (len(simulations) + 2) + len(simulations) for x in range(len(vis_regions))]
# y = [log2_min(norm_elem_rates['gen']['Col-0'][r]) for r in vis_regions]
# ax.bar(x, y, yerr=[low_err, high_err], color=gen_cmap('Col-0'))

# low_err = [log2_min(norm_elem_rates['gen']['Ler-0'][r]) - log2_min(max(0.01, norm_elem_rates['low']['Ler-0'][r])) for r in vis_regions]
# high_err = [log2_min(norm_elem_rates['high']['Ler-0'][r]) - log2_min(norm_elem_rates['gen']['Ler-0'][r]) for r in vis_regions]
# x = [(x + 1) * (len(simulations) + 2) + (len(simulations) + 1) for x in range(len(vis_regions))]
# y = [log2_min(norm_elem_rates['gen']['Ler-0'][r]) for r in vis_regions]
# ax.bar(x, y, yerr=[low_err, high_err], color=gen_cmap('Ler-0'))
    
    
# # ax.set_xticks([(x + 1) * (len(simulations) + 2) + (len(simulations) + 2) / 2 for x in range(len(vis_regions))], vis_regions)
# ax.legend(list(simulations.keys()) + ['observed Col-0 somatic', 'observed Ler-0 germline'])

# xlim = ax.get_xlim()
# ax.hlines(0, -10, 100, color='black')
# ax.set_xlim(xlim)
# ax.set_ylabel('log2fold mutation/polymorphism rate relative to uniform')
# fig.savefig('figs/simulated_mutation_distributions.svg', dpi=300, bbox_inches='tight')

In [ ]:
low_err